# Benchmarks

## Initialize

In [2]:
%load_ext autoreload
%autoreload 2

import os
from tqdm.auto import tqdm
import pathlib

import numpy as np
import pandas as pd
import lifelines
import pandas as pd

from joblib import Parallel, delayed
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")
import shutil

import plotly.express as px
import plotly.graph_objects as go
from plotly.graph_objects import Box

import matplotlib.pyplot as plt
from lifelines import CRCSplineFitter
import warnings
from lifelines.utils import CensoringType

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

from IPython.display import clear_output
import pathlib

from lifelines.utils import concordance_index

In [3]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=1, threads_per_worker=10)
client = Client(cluster)

distributed.diskutils - INFO - Found stale lock file and directory '/home/steinfej/code/21_metabolomics_analysis/Round1/dask-worker-space/worker-xtkuvcb_', purging


In [4]:
node = !hostname
if "sc" in node[0]:
    base_path = "/sc-projects/sc-proj-ukb-cvd"
else: base_path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS"
print(base_path)

project_name = "210714_metabolomics"
#data_path = "/data/analysis/ag-reils/steinfej"
data_pre = f"{base_path}/data/2_datasets_pre/{project_name}"
data_post = f"{base_path}/data/3_datasets_post/{project_name}"

project_label = "21_metabolomics_multitask"
project_path = f"{base_path}/results/projects/{project_label}"
figures_path = f"{project_path}/figures"
data_results_path = f"{project_path}/data"
pathlib.Path(figures_path).mkdir(parents=True, exist_ok=True)
pathlib.Path(data_results_path).mkdir(parents=True, exist_ok=True)

/sc-projects/sc-proj-ukb-cvd


In [5]:
endpoints = [
    #Cardiovascular
    'M_MACE',
    'M_coronary_heart_disease',
    'M_cerebral_stroke',
    'M_peripheral_arterial_disease',
    'M_atrial_fibrillation',
    'M_heart_failure',
    'M_abdominal_aortic_aneurysm',
    'M_venous_thrombosis',
    
    # General IM
    'M_type_2_diabetes',
    'M_liver_disease',
    'M_renal_disease',
    
    # Pulmological
    'M_asthma', 
    'M_chronic_obstructuve_pulmonary_disease',  
    
    # Psychiatric/Neurological
    'M_all_cause_dementia',
    'M_parkinsons_disease',   
    
    # Cancers
    "M_lung_cancer",
    "M_non_melanoma_skin_cancer",
    "M_colon_cancer",
    "M_rectal_cancer",
    "M_prostate_cancer",
    "M_breast_cancer",
    
    # Ophtalmological
    'M_cataracts', 
    'M_glaucoma',
    
    # Traumatology
    'M_fractures',
]

In [6]:
partitions = [str(p) for p in range(0, 22)]
splits = ["train", "valid", "test"]

In [7]:
run = "220126"

In [8]:
!ls {data_results_path}

MET_HRs_210819_metabolomics.feather
MET_HRs_210821_metabolomics.feather
NMR_acetone_IMPUTER.pkl
NMR_clinical_ldl_cholesterol_IMPUTER.pkl
REVISION_220110_attrib_raw.feather
REVISION_220111_temp_umap_shap.feather
add_data_211005.feather
adj_partial_effects_metabolomics_210914.feather
adj_partial_effects_metabolomics_210916.feather
adj_partial_effects_metabolomics_210920.feather
adj_partial_effects_metabolomics_210922A.feather
adj_partial_effects_metabolomics_211005_incl_cancer.feather
adj_partial_effects_metabolomics_211006_improved.feather
adj_partial_effects_metabolomics_211007.feather
adj_partial_effects_metabolomics_220126_apoe4.feather
attrib_raw_REVISION.feather
attrib_sample_wide_211028.feather
attributions_210915.feather
attributions_210916.feather
attributions_210917.feather
attributions_210924.feather
attributions_211008.feather
attributions_211014_Kernelexplainer.feather
attributions_211026.feather
benchmark1000_cindex_subgroups_220106.feather
benchmark100_cindex_subgroups_220

### Load models

In [9]:
project="CardioRS/metabolomics"
#modules = ["CoxPH", "DeepHit", "DeepSurvivalMachine"]
task = "MultiTaskSurvivalTask"
survival_task = "DeepSurvivalMachine"
modules = ["DeepSurvivalMachine"]
datamodules = ["UKBBSurvivalDatamodule"]
#feature_sets = ["AgeSex", "Metabolomics", "AgeSexMetabolomics"]
tag="211006_ds_multitask_cancer_24"
#tag="TaskSpecificExclusion_Debug!"
#https://docs.wandb.ai/ref/app/features/panels/code
import neptune.new as neptune
project = neptune.get_project(project, api_token)
df = project.fetch_runs_table(tag=tag).to_pandas()

In [10]:
log_path = f"{base_path}/results/models/NeptuneLogger"

In [11]:
from pathlib import Path
pred_paths= [f"{log_path}/{Path(*Path(p).parts[-3:])}" for p in df["prediction_path"].to_list() if p==p]

In [12]:
import joblib
def get_df(path): return pd.read_feather(path)#return pd.read_csv(f"{path[:-8]}.csv", index_col=0)
print("Status: ", sum(x is not None for x in pred_paths)/len(pred_paths))
with joblib.parallel_backend('dask'):
    dfs = Parallel(n_jobs=80)(delayed(get_df)(path) for path in tqdm(pred_paths) if path is not None if not pd.isna(path))

Status:  1.0


  0%|          | 0/22 [00:00<?, ?it/s]

In [13]:
preds_all = pd.concat(dfs, axis=0).reset_index(drop=True)

In [14]:
preds_all.query("split=='test'")

,1_1_ft__M_MACE,1_1_Ft__M_MACE,1_1_St__M_MACE,1_2_ft__M_MACE,1_2_Ft__M_MACE,1_2_St__M_MACE,1_3_ft__M_MACE,1_3_Ft__M_MACE,1_3_St__M_MACE,1_4_ft__M_MACE,...,events_M_cataracts,durations_M_glaucoma,events_M_glaucoma,eid,split,partition,module,datamodule,event_names,feature_names
115098,-0.217952,-0.217952,1.217952,-0.217952,-0.217952,1.217952,-0.217952,-0.217952,1.217952,-0.217952,...,0.0,12.301164,0.0,2591257,test,18,ResidualMultiTaskSurvivalTraining,UKBBSurvivalDatamoduleWithExclusions,"['M_MACE_event', 'M_all_cause_dementia_event',...","['NMR_3hydroxybutyrate', 'NMR_acetate', 'NMR_a..."
115099,0.437402,0.437402,0.562598,0.437402,0.437402,0.562598,0.437402,0.437402,0.562598,0.437402,...,1.0,2.954141,1.0,2591268,test,18,ResidualMultiTaskSurvivalTraining,UKBBSurvivalDatamoduleWithExclusions,"['M_MACE_event', 'M_all_cause_dementia_event',...","['NMR_3hydroxybutyrate', 'NMR_acetate', 'NMR_a..."
115100,-0.142463,-0.142463,1.142463,-0.142463,-0.142463,1.142463,-0.142463,-0.142463,1.142463,-0.142463,...,1.0,12.835045,1.0,2591294,test,18,ResidualMultiTaskSurvivalTraining,UKBBSurvivalDatamoduleWithExclusions,"['M_MACE_event', 'M_all_cause_dementia_event',...","['NMR_3hydroxybutyrate', 'NMR_acetate', 'NMR_a..."
115101,0.899744,0.899744,0.100256,0.899744,0.899744,0.100256,0.899744,0.899744,0.100256,0.899744,...,0.0,10.778918,0.0,2591326,test,18,ResidualMultiTaskSurvivalTraining,UKBBSurvivalDatamoduleWithExclusions,"['M_MACE_event', 'M_all_cause_dementia_event',...","['NMR_3hydroxybutyrate', 'NMR_acetate', 'NMR_a..."
115102,-0.577408,-0.577408,1.577409,-0.577408,-0.577408,1.577409,-0.577408,-0.577408,1.577409,-0.577408,...,0.0,13.659138,0.0,2591446,test,18,ResidualMultiTaskSurvivalTraining,UKBBSurvivalDatamoduleWithExclusions,"['M_MACE_event', 'M_all_cause_dementia_event',...","['NMR_3hydroxybutyrate', 'NMR_acetate', 'NMR_a..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2595577,-0.728889,-0.728889,1.728889,-0.728889,-0.728889,1.728889,-0.728889,-0.728889,1.728889,-0.728889,...,0.0,13.327858,0.0,6024939,test,21,ResidualMultiTaskSurvivalTraining,UKBBSurvivalDatamoduleWithExclusions,"['M_MACE_event', 'M_all_cause_dementia_event',...","['NMR_3hydroxybutyrate', 'NMR_acetate', 'NMR_a..."
2595578,-0.776245,-0.776245,1.776245,-0.776245,-0.776245,1.776245,-0.776245,-0.776245,1.776245,-0.776245,...,0.0,12.139630,0.0,6024995,test,21,ResidualMultiTaskSurvivalTraining,UKBBSurvivalDatamoduleWithExclusions,"['M_MACE_event', 'M_all_cause_dementia_event',...","['NMR_3hydroxybutyrate', 'NMR_acetate', 'NMR_a..."
2595579,-0.051542,-0.051542,1.051542,-0.051542,-0.051542,1.051542,-0.051542,-0.051542,1.051542,-0.051542,...,0.0,11.685147,0.0,6025042,test,21,ResidualMultiTaskSurvivalTraining,UKBBSurvivalDatamoduleWithExclusions,"['M_MACE_event', 'M_all_cause_dementia_event',...","['NMR_3hydroxybutyrate', 'NMR_acetate', 'NMR_a..."
2595580,0.401364,0.401364,0.598636,0.401364,0.401364,0.598636,0.401364,0.401364,0.598636,0.401364,...,0.0,11.649555,0.0,6025116,test,21,ResidualMultiTaskSurvivalTraining,UKBBSurvivalDatamoduleWithExclusions,"['M_MACE_event', 'M_all_cause_dementia_event',...","['NMR_3hydroxybutyrate', 'NMR_acetate', 'NMR_a..."


In [15]:
#preds_all.to_feather(f"{data_results_path}/preds_all.feather")

In [16]:
def get_feature_string(d):
    l = []
    for k, v in eval(d).items():
        for i, j in v.items():
            for m in j:
                if not m in l: l.extend(j)
    return repr(l)

def clean_df(df, predictions):
   #feature_df = df[[\"parameters/feature_set\", \"parameters/features_yaml\"]].drop_duplicates().dropna()\n",
   #feature_map ={row[\"parameters/features_yaml\"]: row[\"parameters/feature_set\"] for i, row in feature_df.iterrows()}\n",
    predictions["features"] = predictions["features_names"].map(feature_map)
    return predictions

In [17]:
def clean_df(df, predictions):
#feature_df = df[[\"parameters/feature_set\", \"parameters/features_yaml\"]].drop_duplicates().dropna()\n",
    feature_map ={"['age_at_recruitment', 'sex', 'NMR_3hydroxybutyrate', 'NMR_acetate', 'NMR_acetoacetate', 'NMR_acetone', 'NMR_alanine', 'NMR_albumin', 'NMR_apolipoprotein_a1', 'NMR_apolipoprotein_b', 'NMR_average_diameter_for_hdl_particles', 'NMR_average_diameter_for_ldl_particles', 'NMR_average_diameter_for_vldl_particles', 'NMR_cholesterol_in_chylomicrons_and_extremely_large_vldl', 'NMR_cholesterol_in_idl', 'NMR_cholesterol_in_large_hdl', 'NMR_cholesterol_in_large_ldl', 'NMR_cholesterol_in_large_vldl', 'NMR_cholesterol_in_medium_hdl', 'NMR_cholesterol_in_medium_ldl', 'NMR_cholesterol_in_medium_vldl', 'NMR_cholesterol_in_small_hdl', 'NMR_cholesterol_in_small_ldl', 'NMR_cholesterol_in_small_vldl', 'NMR_cholesterol_in_very_large_hdl', 'NMR_cholesterol_in_very_large_vldl', 'NMR_cholesterol_in_very_small_vldl', 'NMR_cholesteryl_esters_in_chylomicrons_and_extremely_large_vldl', 'NMR_cholesteryl_esters_in_hdl', 'NMR_cholesteryl_esters_in_idl', 'NMR_cholesteryl_esters_in_ldl', 'NMR_cholesteryl_esters_in_large_hdl', 'NMR_cholesteryl_esters_in_large_ldl', 'NMR_cholesteryl_esters_in_large_vldl', 'NMR_cholesteryl_esters_in_medium_hdl', 'NMR_cholesteryl_esters_in_medium_ldl', 'NMR_cholesteryl_esters_in_medium_vldl', 'NMR_cholesteryl_esters_in_small_hdl', 'NMR_cholesteryl_esters_in_small_ldl', 'NMR_cholesteryl_esters_in_small_vldl', 'NMR_cholesteryl_esters_in_vldl', 'NMR_cholesteryl_esters_in_very_large_hdl', 'NMR_cholesteryl_esters_in_very_large_vldl', 'NMR_cholesteryl_esters_in_very_small_vldl', 'NMR_citrate', 'NMR_clinical_ldl_cholesterol', 'NMR_concentration_of_chylomicrons_and_extremely_large_vldl_particles', 'NMR_concentration_of_hdl_particles', 'NMR_concentration_of_idl_particles', 'NMR_concentration_of_ldl_particles', 'NMR_concentration_of_large_hdl_particles', 'NMR_concentration_of_large_ldl_particles', 'NMR_concentration_of_large_vldl_particles', 'NMR_concentration_of_medium_hdl_particles', 'NMR_concentration_of_medium_ldl_particles', 'NMR_concentration_of_medium_vldl_particles', 'NMR_concentration_of_small_hdl_particles', 'NMR_concentration_of_small_ldl_particles', 'NMR_concentration_of_small_vldl_particles', 'NMR_concentration_of_vldl_particles', 'NMR_concentration_of_very_large_hdl_particles', 'NMR_concentration_of_very_large_vldl_particles', 'NMR_concentration_of_very_small_vldl_particles', 'NMR_creatinine', 'NMR_degree_of_unsaturation', 'NMR_docosahexaenoic_acid', 'NMR_free_cholesterol_in_chylomicrons_and_extremely_large_vldl', 'NMR_free_cholesterol_in_hdl', 'NMR_free_cholesterol_in_idl', 'NMR_free_cholesterol_in_ldl', 'NMR_free_cholesterol_in_large_hdl', 'NMR_free_cholesterol_in_large_ldl', 'NMR_free_cholesterol_in_large_vldl', 'NMR_free_cholesterol_in_medium_hdl', 'NMR_free_cholesterol_in_medium_ldl', 'NMR_free_cholesterol_in_medium_vldl', 'NMR_free_cholesterol_in_small_hdl', 'NMR_free_cholesterol_in_small_ldl', 'NMR_free_cholesterol_in_small_vldl', 'NMR_free_cholesterol_in_vldl', 'NMR_free_cholesterol_in_very_large_hdl', 'NMR_free_cholesterol_in_very_large_vldl', 'NMR_free_cholesterol_in_very_small_vldl', 'NMR_glucose', 'NMR_glutamine', 'NMR_glycine', 'NMR_glycoprotein_acetyls', 'NMR_hdl_cholesterol', 'NMR_histidine', 'NMR_isoleucine', 'NMR_ldl_cholesterol', 'NMR_lactate', 'NMR_leucine', 'NMR_linoleic_acid', 'NMR_monounsaturated_fatty_acids', 'NMR_omega3_fatty_acids', 'NMR_omega6_fatty_acids', 'NMR_phenylalanine', 'NMR_phosphatidylcholines', 'NMR_phosphoglycerides', 'NMR_phospholipids_in_chylomicrons_and_extremely_large_vldl', 'NMR_phospholipids_in_hdl', 'NMR_phospholipids_in_idl', 'NMR_phospholipids_in_ldl', 'NMR_phospholipids_in_large_hdl', 'NMR_phospholipids_in_large_ldl', 'NMR_phospholipids_in_large_vldl', 'NMR_phospholipids_in_medium_hdl', 'NMR_phospholipids_in_medium_ldl', 'NMR_phospholipids_in_medium_vldl', 'NMR_phospholipids_in_small_hdl', 'NMR_phospholipids_in_small_ldl', 'NMR_phospholipids_in_small_vldl', 'NMR_phospholipids_in_vldl', 'NMR_phospholipids_in_very_large_hdl', 'NMR_phospholipids_in_very_large_vldl', 'NMR_phospholipids_in_very_small_vldl', 'NMR_polyunsaturated_fatty_acids', 'NMR_pyruvate', 'NMR_remnant_cholesterol_nonhdl_nonldl_cholesterol', 'NMR_saturated_fatty_acids', 'NMR_sphingomyelins', 'NMR_total_cholesterol', 'NMR_total_cholesterol_minus_hdlc', 'NMR_total_cholines', 'NMR_total_concentration_of_branchedchain_amino_acids_leucine_isoleucine_valine', 'NMR_total_concentration_of_lipoprotein_particles', 'NMR_total_esterified_cholesterol', 'NMR_total_fatty_acids', 'NMR_total_free_cholesterol', 'NMR_total_lipids_in_chylomicrons_and_extremely_large_vldl', 'NMR_total_lipids_in_hdl', 'NMR_total_lipids_in_idl', 'NMR_total_lipids_in_ldl', 'NMR_total_lipids_in_large_hdl', 'NMR_total_lipids_in_large_ldl', 'NMR_total_lipids_in_large_vldl', 'NMR_total_lipids_in_lipoprotein_particles', 'NMR_total_lipids_in_medium_hdl', 'NMR_total_lipids_in_medium_ldl', 'NMR_total_lipids_in_medium_vldl', 'NMR_total_lipids_in_small_hdl', 'NMR_total_lipids_in_small_ldl', 'NMR_total_lipids_in_small_vldl', 'NMR_total_lipids_in_vldl', 'NMR_total_lipids_in_very_large_hdl', 'NMR_total_lipids_in_very_large_vldl', 'NMR_total_lipids_in_very_small_vldl', 'NMR_total_phospholipids_in_lipoprotein_particles', 'NMR_total_triglycerides', 'NMR_triglycerides_in_chylomicrons_and_extremely_large_vldl', 'NMR_triglycerides_in_hdl', 'NMR_triglycerides_in_idl', 'NMR_triglycerides_in_ldl', 'NMR_triglycerides_in_large_hdl', 'NMR_triglycerides_in_large_ldl', 'NMR_triglycerides_in_large_vldl', 'NMR_triglycerides_in_medium_hdl', 'NMR_triglycerides_in_medium_ldl', 'NMR_triglycerides_in_medium_vldl', 'NMR_triglycerides_in_small_hdl', 'NMR_triglycerides_in_small_ldl', 'NMR_triglycerides_in_small_vldl', 'NMR_triglycerides_in_vldl', 'NMR_triglycerides_in_very_large_hdl', 'NMR_triglycerides_in_very_large_vldl', 'NMR_triglycerides_in_very_small_vldl', 'NMR_tyrosine', 'NMR_vldl_cholesterol', 'NMR_valine']":"AgeSexMetabolomics",
                  "['NMR_3hydroxybutyrate', 'NMR_acetate', 'NMR_acetoacetate', 'NMR_acetone', 'NMR_alanine', 'NMR_albumin', 'NMR_apolipoprotein_a1', 'NMR_apolipoprotein_b', 'NMR_average_diameter_for_hdl_particles', 'NMR_average_diameter_for_ldl_particles', 'NMR_average_diameter_for_vldl_particles', 'NMR_cholesterol_in_chylomicrons_and_extremely_large_vldl', 'NMR_cholesterol_in_idl', 'NMR_cholesterol_in_large_hdl', 'NMR_cholesterol_in_large_ldl', 'NMR_cholesterol_in_large_vldl', 'NMR_cholesterol_in_medium_hdl', 'NMR_cholesterol_in_medium_ldl', 'NMR_cholesterol_in_medium_vldl', 'NMR_cholesterol_in_small_hdl', 'NMR_cholesterol_in_small_ldl', 'NMR_cholesterol_in_small_vldl', 'NMR_cholesterol_in_very_large_hdl', 'NMR_cholesterol_in_very_large_vldl', 'NMR_cholesterol_in_very_small_vldl', 'NMR_cholesteryl_esters_in_chylomicrons_and_extremely_large_vldl', 'NMR_cholesteryl_esters_in_hdl', 'NMR_cholesteryl_esters_in_idl', 'NMR_cholesteryl_esters_in_ldl', 'NMR_cholesteryl_esters_in_large_hdl', 'NMR_cholesteryl_esters_in_large_ldl', 'NMR_cholesteryl_esters_in_large_vldl', 'NMR_cholesteryl_esters_in_medium_hdl', 'NMR_cholesteryl_esters_in_medium_ldl', 'NMR_cholesteryl_esters_in_medium_vldl', 'NMR_cholesteryl_esters_in_small_hdl', 'NMR_cholesteryl_esters_in_small_ldl', 'NMR_cholesteryl_esters_in_small_vldl', 'NMR_cholesteryl_esters_in_vldl', 'NMR_cholesteryl_esters_in_very_large_hdl', 'NMR_cholesteryl_esters_in_very_large_vldl', 'NMR_cholesteryl_esters_in_very_small_vldl', 'NMR_citrate', 'NMR_clinical_ldl_cholesterol', 'NMR_concentration_of_chylomicrons_and_extremely_large_vldl_particles', 'NMR_concentration_of_hdl_particles', 'NMR_concentration_of_idl_particles', 'NMR_concentration_of_ldl_particles', 'NMR_concentration_of_large_hdl_particles', 'NMR_concentration_of_large_ldl_particles', 'NMR_concentration_of_large_vldl_particles', 'NMR_concentration_of_medium_hdl_particles', 'NMR_concentration_of_medium_ldl_particles', 'NMR_concentration_of_medium_vldl_particles', 'NMR_concentration_of_small_hdl_particles', 'NMR_concentration_of_small_ldl_particles', 'NMR_concentration_of_small_vldl_particles', 'NMR_concentration_of_vldl_particles', 'NMR_concentration_of_very_large_hdl_particles', 'NMR_concentration_of_very_large_vldl_particles', 'NMR_concentration_of_very_small_vldl_particles', 'NMR_creatinine', 'NMR_degree_of_unsaturation', 'NMR_docosahexaenoic_acid', 'NMR_free_cholesterol_in_chylomicrons_and_extremely_large_vldl', 'NMR_free_cholesterol_in_hdl', 'NMR_free_cholesterol_in_idl', 'NMR_free_cholesterol_in_ldl', 'NMR_free_cholesterol_in_large_hdl', 'NMR_free_cholesterol_in_large_ldl', 'NMR_free_cholesterol_in_large_vldl', 'NMR_free_cholesterol_in_medium_hdl', 'NMR_free_cholesterol_in_medium_ldl', 'NMR_free_cholesterol_in_medium_vldl', 'NMR_free_cholesterol_in_small_hdl', 'NMR_free_cholesterol_in_small_ldl', 'NMR_free_cholesterol_in_small_vldl', 'NMR_free_cholesterol_in_vldl', 'NMR_free_cholesterol_in_very_large_hdl', 'NMR_free_cholesterol_in_very_large_vldl', 'NMR_free_cholesterol_in_very_small_vldl', 'NMR_glucose', 'NMR_glutamine', 'NMR_glycine', 'NMR_glycoprotein_acetyls', 'NMR_hdl_cholesterol', 'NMR_histidine', 'NMR_isoleucine', 'NMR_ldl_cholesterol', 'NMR_lactate', 'NMR_leucine', 'NMR_linoleic_acid', 'NMR_monounsaturated_fatty_acids', 'NMR_omega3_fatty_acids', 'NMR_omega6_fatty_acids', 'NMR_phenylalanine', 'NMR_phosphatidylcholines', 'NMR_phosphoglycerides', 'NMR_phospholipids_in_chylomicrons_and_extremely_large_vldl', 'NMR_phospholipids_in_hdl', 'NMR_phospholipids_in_idl', 'NMR_phospholipids_in_ldl', 'NMR_phospholipids_in_large_hdl', 'NMR_phospholipids_in_large_ldl', 'NMR_phospholipids_in_large_vldl', 'NMR_phospholipids_in_medium_hdl', 'NMR_phospholipids_in_medium_ldl', 'NMR_phospholipids_in_medium_vldl', 'NMR_phospholipids_in_small_hdl', 'NMR_phospholipids_in_small_ldl', 'NMR_phospholipids_in_small_vldl', 'NMR_phospholipids_in_vldl', 'NMR_phospholipids_in_very_large_hdl', 'NMR_phospholipids_in_very_large_vldl', 'NMR_phospholipids_in_very_small_vldl', 'NMR_polyunsaturated_fatty_acids', 'NMR_pyruvate', 'NMR_remnant_cholesterol_nonhdl_nonldl_cholesterol', 'NMR_saturated_fatty_acids', 'NMR_sphingomyelins', 'NMR_total_cholesterol', 'NMR_total_cholesterol_minus_hdlc', 'NMR_total_cholines', 'NMR_total_concentration_of_branchedchain_amino_acids_leucine_isoleucine_valine', 'NMR_total_concentration_of_lipoprotein_particles', 'NMR_total_esterified_cholesterol', 'NMR_total_fatty_acids', 'NMR_total_free_cholesterol', 'NMR_total_lipids_in_chylomicrons_and_extremely_large_vldl', 'NMR_total_lipids_in_hdl', 'NMR_total_lipids_in_idl', 'NMR_total_lipids_in_ldl', 'NMR_total_lipids_in_large_hdl', 'NMR_total_lipids_in_large_ldl', 'NMR_total_lipids_in_large_vldl', 'NMR_total_lipids_in_lipoprotein_particles', 'NMR_total_lipids_in_medium_hdl', 'NMR_total_lipids_in_medium_ldl', 'NMR_total_lipids_in_medium_vldl', 'NMR_total_lipids_in_small_hdl', 'NMR_total_lipids_in_small_ldl', 'NMR_total_lipids_in_small_vldl', 'NMR_total_lipids_in_vldl', 'NMR_total_lipids_in_very_large_hdl', 'NMR_total_lipids_in_very_large_vldl', 'NMR_total_lipids_in_very_small_vldl', 'NMR_total_phospholipids_in_lipoprotein_particles', 'NMR_total_triglycerides', 'NMR_triglycerides_in_chylomicrons_and_extremely_large_vldl', 'NMR_triglycerides_in_hdl', 'NMR_triglycerides_in_idl', 'NMR_triglycerides_in_ldl', 'NMR_triglycerides_in_large_hdl', 'NMR_triglycerides_in_large_ldl', 'NMR_triglycerides_in_large_vldl', 'NMR_triglycerides_in_medium_hdl', 'NMR_triglycerides_in_medium_ldl', 'NMR_triglycerides_in_medium_vldl', 'NMR_triglycerides_in_small_hdl', 'NMR_triglycerides_in_small_ldl', 'NMR_triglycerides_in_small_vldl', 'NMR_triglycerides_in_vldl', 'NMR_triglycerides_in_very_large_hdl', 'NMR_triglycerides_in_very_large_vldl', 'NMR_triglycerides_in_very_small_vldl', 'NMR_tyrosine', 'NMR_vldl_cholesterol', 'NMR_valine']":"Metabolomics"}
    predictions["features"] = predictions["feature_names"].map(feature_map)
    return predictions

In [18]:
preds_all_cleaned = clean_df(df, preds_all)

In [19]:
def clean_df(df, predictions):
    #feature_df = df[["parameters/feature_set", "parameters/features_yaml"]].drop_duplicates().dropna()
    feature_map ={"['age_at_recruitment', 'sex', 'NMR_3hydroxybutyrate', 'NMR_acetate', 'NMR_acetoacetate', 'NMR_acetone', 'NMR_alanine', 'NMR_albumin', 'NMR_apolipoprotein_a1', 'NMR_apolipoprotein_b', 'NMR_average_diameter_for_hdl_particles', 'NMR_average_diameter_for_ldl_particles', 'NMR_average_diameter_for_vldl_particles', 'NMR_cholesterol_in_chylomicrons_and_extremely_large_vldl', 'NMR_cholesterol_in_idl', 'NMR_cholesterol_in_large_hdl', 'NMR_cholesterol_in_large_ldl', 'NMR_cholesterol_in_large_vldl', 'NMR_cholesterol_in_medium_hdl', 'NMR_cholesterol_in_medium_ldl', 'NMR_cholesterol_in_medium_vldl', 'NMR_cholesterol_in_small_hdl', 'NMR_cholesterol_in_small_ldl', 'NMR_cholesterol_in_small_vldl', 'NMR_cholesterol_in_very_large_hdl', 'NMR_cholesterol_in_very_large_vldl', 'NMR_cholesterol_in_very_small_vldl', 'NMR_cholesteryl_esters_in_chylomicrons_and_extremely_large_vldl', 'NMR_cholesteryl_esters_in_hdl', 'NMR_cholesteryl_esters_in_idl', 'NMR_cholesteryl_esters_in_ldl', 'NMR_cholesteryl_esters_in_large_hdl', 'NMR_cholesteryl_esters_in_large_ldl', 'NMR_cholesteryl_esters_in_large_vldl', 'NMR_cholesteryl_esters_in_medium_hdl', 'NMR_cholesteryl_esters_in_medium_ldl', 'NMR_cholesteryl_esters_in_medium_vldl', 'NMR_cholesteryl_esters_in_small_hdl', 'NMR_cholesteryl_esters_in_small_ldl', 'NMR_cholesteryl_esters_in_small_vldl', 'NMR_cholesteryl_esters_in_vldl', 'NMR_cholesteryl_esters_in_very_large_hdl', 'NMR_cholesteryl_esters_in_very_large_vldl', 'NMR_cholesteryl_esters_in_very_small_vldl', 'NMR_citrate', 'NMR_clinical_ldl_cholesterol', 'NMR_concentration_of_chylomicrons_and_extremely_large_vldl_particles', 'NMR_concentration_of_hdl_particles', 'NMR_concentration_of_idl_particles', 'NMR_concentration_of_ldl_particles', 'NMR_concentration_of_large_hdl_particles', 'NMR_concentration_of_large_ldl_particles', 'NMR_concentration_of_large_vldl_particles', 'NMR_concentration_of_medium_hdl_particles', 'NMR_concentration_of_medium_ldl_particles', 'NMR_concentration_of_medium_vldl_particles', 'NMR_concentration_of_small_hdl_particles', 'NMR_concentration_of_small_ldl_particles', 'NMR_concentration_of_small_vldl_particles', 'NMR_concentration_of_vldl_particles', 'NMR_concentration_of_very_large_hdl_particles', 'NMR_concentration_of_very_large_vldl_particles', 'NMR_concentration_of_very_small_vldl_particles', 'NMR_creatinine', 'NMR_degree_of_unsaturation', 'NMR_docosahexaenoic_acid', 'NMR_free_cholesterol_in_chylomicrons_and_extremely_large_vldl', 'NMR_free_cholesterol_in_hdl', 'NMR_free_cholesterol_in_idl', 'NMR_free_cholesterol_in_ldl', 'NMR_free_cholesterol_in_large_hdl', 'NMR_free_cholesterol_in_large_ldl', 'NMR_free_cholesterol_in_large_vldl', 'NMR_free_cholesterol_in_medium_hdl', 'NMR_free_cholesterol_in_medium_ldl', 'NMR_free_cholesterol_in_medium_vldl', 'NMR_free_cholesterol_in_small_hdl', 'NMR_free_cholesterol_in_small_ldl', 'NMR_free_cholesterol_in_small_vldl', 'NMR_free_cholesterol_in_vldl', 'NMR_free_cholesterol_in_very_large_hdl', 'NMR_free_cholesterol_in_very_large_vldl', 'NMR_free_cholesterol_in_very_small_vldl', 'NMR_glucose', 'NMR_glutamine', 'NMR_glycine', 'NMR_glycoprotein_acetyls', 'NMR_hdl_cholesterol', 'NMR_histidine', 'NMR_isoleucine', 'NMR_ldl_cholesterol', 'NMR_lactate', 'NMR_leucine', 'NMR_linoleic_acid', 'NMR_monounsaturated_fatty_acids', 'NMR_omega3_fatty_acids', 'NMR_omega6_fatty_acids', 'NMR_phenylalanine', 'NMR_phosphatidylcholines', 'NMR_phosphoglycerides', 'NMR_phospholipids_in_chylomicrons_and_extremely_large_vldl', 'NMR_phospholipids_in_hdl', 'NMR_phospholipids_in_idl', 'NMR_phospholipids_in_ldl', 'NMR_phospholipids_in_large_hdl', 'NMR_phospholipids_in_large_ldl', 'NMR_phospholipids_in_large_vldl', 'NMR_phospholipids_in_medium_hdl', 'NMR_phospholipids_in_medium_ldl', 'NMR_phospholipids_in_medium_vldl', 'NMR_phospholipids_in_small_hdl', 'NMR_phospholipids_in_small_ldl', 'NMR_phospholipids_in_small_vldl', 'NMR_phospholipids_in_vldl', 'NMR_phospholipids_in_very_large_hdl', 'NMR_phospholipids_in_very_large_vldl', 'NMR_phospholipids_in_very_small_vldl', 'NMR_polyunsaturated_fatty_acids', 'NMR_pyruvate', 'NMR_remnant_cholesterol_nonhdl_nonldl_cholesterol', 'NMR_saturated_fatty_acids', 'NMR_sphingomyelins', 'NMR_total_cholesterol', 'NMR_total_cholesterol_minus_hdlc', 'NMR_total_cholines', 'NMR_total_concentration_of_branchedchain_amino_acids_leucine_isoleucine_valine', 'NMR_total_concentration_of_lipoprotein_particles', 'NMR_total_esterified_cholesterol', 'NMR_total_fatty_acids', 'NMR_total_free_cholesterol', 'NMR_total_lipids_in_chylomicrons_and_extremely_large_vldl', 'NMR_total_lipids_in_hdl', 'NMR_total_lipids_in_idl', 'NMR_total_lipids_in_ldl', 'NMR_total_lipids_in_large_hdl', 'NMR_total_lipids_in_large_ldl', 'NMR_total_lipids_in_large_vldl', 'NMR_total_lipids_in_lipoprotein_particles', 'NMR_total_lipids_in_medium_hdl', 'NMR_total_lipids_in_medium_ldl', 'NMR_total_lipids_in_medium_vldl', 'NMR_total_lipids_in_small_hdl', 'NMR_total_lipids_in_small_ldl', 'NMR_total_lipids_in_small_vldl', 'NMR_total_lipids_in_vldl', 'NMR_total_lipids_in_very_large_hdl', 'NMR_total_lipids_in_very_large_vldl', 'NMR_total_lipids_in_very_small_vldl', 'NMR_total_phospholipids_in_lipoprotein_particles', 'NMR_total_triglycerides', 'NMR_triglycerides_in_chylomicrons_and_extremely_large_vldl', 'NMR_triglycerides_in_hdl', 'NMR_triglycerides_in_idl', 'NMR_triglycerides_in_ldl', 'NMR_triglycerides_in_large_hdl', 'NMR_triglycerides_in_large_ldl', 'NMR_triglycerides_in_large_vldl', 'NMR_triglycerides_in_medium_hdl', 'NMR_triglycerides_in_medium_ldl', 'NMR_triglycerides_in_medium_vldl', 'NMR_triglycerides_in_small_hdl', 'NMR_triglycerides_in_small_ldl', 'NMR_triglycerides_in_small_vldl', 'NMR_triglycerides_in_vldl', 'NMR_triglycerides_in_very_large_hdl', 'NMR_triglycerides_in_very_large_vldl', 'NMR_triglycerides_in_very_small_vldl', 'NMR_tyrosine', 'NMR_vldl_cholesterol', 'NMR_valine']":"AgeSexMetabolomics",
                  "['NMR_3hydroxybutyrate', 'NMR_acetate', 'NMR_acetoacetate', 'NMR_acetone', 'NMR_alanine', 'NMR_albumin', 'NMR_apolipoprotein_a1', 'NMR_apolipoprotein_b', 'NMR_average_diameter_for_hdl_particles', 'NMR_average_diameter_for_ldl_particles', 'NMR_average_diameter_for_vldl_particles', 'NMR_cholesterol_in_chylomicrons_and_extremely_large_vldl', 'NMR_cholesterol_in_idl', 'NMR_cholesterol_in_large_hdl', 'NMR_cholesterol_in_large_ldl', 'NMR_cholesterol_in_large_vldl', 'NMR_cholesterol_in_medium_hdl', 'NMR_cholesterol_in_medium_ldl', 'NMR_cholesterol_in_medium_vldl', 'NMR_cholesterol_in_small_hdl', 'NMR_cholesterol_in_small_ldl', 'NMR_cholesterol_in_small_vldl', 'NMR_cholesterol_in_very_large_hdl', 'NMR_cholesterol_in_very_large_vldl', 'NMR_cholesterol_in_very_small_vldl', 'NMR_cholesteryl_esters_in_chylomicrons_and_extremely_large_vldl', 'NMR_cholesteryl_esters_in_hdl', 'NMR_cholesteryl_esters_in_idl', 'NMR_cholesteryl_esters_in_ldl', 'NMR_cholesteryl_esters_in_large_hdl', 'NMR_cholesteryl_esters_in_large_ldl', 'NMR_cholesteryl_esters_in_large_vldl', 'NMR_cholesteryl_esters_in_medium_hdl', 'NMR_cholesteryl_esters_in_medium_ldl', 'NMR_cholesteryl_esters_in_medium_vldl', 'NMR_cholesteryl_esters_in_small_hdl', 'NMR_cholesteryl_esters_in_small_ldl', 'NMR_cholesteryl_esters_in_small_vldl', 'NMR_cholesteryl_esters_in_vldl', 'NMR_cholesteryl_esters_in_very_large_hdl', 'NMR_cholesteryl_esters_in_very_large_vldl', 'NMR_cholesteryl_esters_in_very_small_vldl', 'NMR_citrate', 'NMR_clinical_ldl_cholesterol', 'NMR_concentration_of_chylomicrons_and_extremely_large_vldl_particles', 'NMR_concentration_of_hdl_particles', 'NMR_concentration_of_idl_particles', 'NMR_concentration_of_ldl_particles', 'NMR_concentration_of_large_hdl_particles', 'NMR_concentration_of_large_ldl_particles', 'NMR_concentration_of_large_vldl_particles', 'NMR_concentration_of_medium_hdl_particles', 'NMR_concentration_of_medium_ldl_particles', 'NMR_concentration_of_medium_vldl_particles', 'NMR_concentration_of_small_hdl_particles', 'NMR_concentration_of_small_ldl_particles', 'NMR_concentration_of_small_vldl_particles', 'NMR_concentration_of_vldl_particles', 'NMR_concentration_of_very_large_hdl_particles', 'NMR_concentration_of_very_large_vldl_particles', 'NMR_concentration_of_very_small_vldl_particles', 'NMR_creatinine', 'NMR_degree_of_unsaturation', 'NMR_docosahexaenoic_acid', 'NMR_free_cholesterol_in_chylomicrons_and_extremely_large_vldl', 'NMR_free_cholesterol_in_hdl', 'NMR_free_cholesterol_in_idl', 'NMR_free_cholesterol_in_ldl', 'NMR_free_cholesterol_in_large_hdl', 'NMR_free_cholesterol_in_large_ldl', 'NMR_free_cholesterol_in_large_vldl', 'NMR_free_cholesterol_in_medium_hdl', 'NMR_free_cholesterol_in_medium_ldl', 'NMR_free_cholesterol_in_medium_vldl', 'NMR_free_cholesterol_in_small_hdl', 'NMR_free_cholesterol_in_small_ldl', 'NMR_free_cholesterol_in_small_vldl', 'NMR_free_cholesterol_in_vldl', 'NMR_free_cholesterol_in_very_large_hdl', 'NMR_free_cholesterol_in_very_large_vldl', 'NMR_free_cholesterol_in_very_small_vldl', 'NMR_glucose', 'NMR_glutamine', 'NMR_glycine', 'NMR_glycoprotein_acetyls', 'NMR_hdl_cholesterol', 'NMR_histidine', 'NMR_isoleucine', 'NMR_ldl_cholesterol', 'NMR_lactate', 'NMR_leucine', 'NMR_linoleic_acid', 'NMR_monounsaturated_fatty_acids', 'NMR_omega3_fatty_acids', 'NMR_omega6_fatty_acids', 'NMR_phenylalanine', 'NMR_phosphatidylcholines', 'NMR_phosphoglycerides', 'NMR_phospholipids_in_chylomicrons_and_extremely_large_vldl', 'NMR_phospholipids_in_hdl', 'NMR_phospholipids_in_idl', 'NMR_phospholipids_in_ldl', 'NMR_phospholipids_in_large_hdl', 'NMR_phospholipids_in_large_ldl', 'NMR_phospholipids_in_large_vldl', 'NMR_phospholipids_in_medium_hdl', 'NMR_phospholipids_in_medium_ldl', 'NMR_phospholipids_in_medium_vldl', 'NMR_phospholipids_in_small_hdl', 'NMR_phospholipids_in_small_ldl', 'NMR_phospholipids_in_small_vldl', 'NMR_phospholipids_in_vldl', 'NMR_phospholipids_in_very_large_hdl', 'NMR_phospholipids_in_very_large_vldl', 'NMR_phospholipids_in_very_small_vldl', 'NMR_polyunsaturated_fatty_acids', 'NMR_pyruvate', 'NMR_remnant_cholesterol_nonhdl_nonldl_cholesterol', 'NMR_saturated_fatty_acids', 'NMR_sphingomyelins', 'NMR_total_cholesterol', 'NMR_total_cholesterol_minus_hdlc', 'NMR_total_cholines', 'NMR_total_concentration_of_branchedchain_amino_acids_leucine_isoleucine_valine', 'NMR_total_concentration_of_lipoprotein_particles', 'NMR_total_esterified_cholesterol', 'NMR_total_fatty_acids', 'NMR_total_free_cholesterol', 'NMR_total_lipids_in_chylomicrons_and_extremely_large_vldl', 'NMR_total_lipids_in_hdl', 'NMR_total_lipids_in_idl', 'NMR_total_lipids_in_ldl', 'NMR_total_lipids_in_large_hdl', 'NMR_total_lipids_in_large_ldl', 'NMR_total_lipids_in_large_vldl', 'NMR_total_lipids_in_lipoprotein_particles', 'NMR_total_lipids_in_medium_hdl', 'NMR_total_lipids_in_medium_ldl', 'NMR_total_lipids_in_medium_vldl', 'NMR_total_lipids_in_small_hdl', 'NMR_total_lipids_in_small_ldl', 'NMR_total_lipids_in_small_vldl', 'NMR_total_lipids_in_vldl', 'NMR_total_lipids_in_very_large_hdl', 'NMR_total_lipids_in_very_large_vldl', 'NMR_total_lipids_in_very_small_vldl', 'NMR_total_phospholipids_in_lipoprotein_particles', 'NMR_total_triglycerides', 'NMR_triglycerides_in_chylomicrons_and_extremely_large_vldl', 'NMR_triglycerides_in_hdl', 'NMR_triglycerides_in_idl', 'NMR_triglycerides_in_ldl', 'NMR_triglycerides_in_large_hdl', 'NMR_triglycerides_in_large_ldl', 'NMR_triglycerides_in_large_vldl', 'NMR_triglycerides_in_medium_hdl', 'NMR_triglycerides_in_medium_ldl', 'NMR_triglycerides_in_medium_vldl', 'NMR_triglycerides_in_small_hdl', 'NMR_triglycerides_in_small_ldl', 'NMR_triglycerides_in_small_vldl', 'NMR_triglycerides_in_vldl', 'NMR_triglycerides_in_very_large_hdl', 'NMR_triglycerides_in_very_large_vldl', 'NMR_triglycerides_in_very_small_vldl', 'NMR_tyrosine', 'NMR_vldl_cholesterol', 'NMR_valine']":"Metabolomics"}
    predictions["features"] = predictions["feature_names"].map(feature_map)
    return predictions

In [20]:
preds_all_cleaned = clean_df(df, preds_all)

In [21]:
preds_all_cleaned

,1_1_ft__M_MACE,1_1_Ft__M_MACE,1_1_St__M_MACE,1_2_ft__M_MACE,1_2_Ft__M_MACE,1_2_St__M_MACE,1_3_ft__M_MACE,1_3_Ft__M_MACE,1_3_St__M_MACE,1_4_ft__M_MACE,...,durations_M_glaucoma,events_M_glaucoma,eid,split,partition,module,datamodule,event_names,feature_names,features
0,0.487153,0.487153,0.512847,0.487153,0.487153,0.512847,0.487153,0.487153,0.512847,0.487153,...,13.429158,0.0,1000084,train,18,ResidualMultiTaskSurvivalTraining,UKBBSurvivalDatamoduleWithExclusions,"['M_MACE_event', 'M_all_cause_dementia_event',...","['NMR_3hydroxybutyrate', 'NMR_acetate', 'NMR_a...",Metabolomics
1,-0.798227,-0.798227,1.798227,-0.798227,-0.798227,1.798227,-0.798227,-0.798227,1.798227,-0.798227,...,12.591375,0.0,1000107,train,18,ResidualMultiTaskSurvivalTraining,UKBBSurvivalDatamoduleWithExclusions,"['M_MACE_event', 'M_all_cause_dementia_event',...","['NMR_3hydroxybutyrate', 'NMR_acetate', 'NMR_a...",Metabolomics
2,-0.999625,-0.999625,1.999625,-0.999625,-0.999625,1.999625,-0.999625,-0.999625,1.999625,-0.999625,...,11.110198,0.0,1000128,train,18,ResidualMultiTaskSurvivalTraining,UKBBSurvivalDatamoduleWithExclusions,"['M_MACE_event', 'M_all_cause_dementia_event',...","['NMR_3hydroxybutyrate', 'NMR_acetate', 'NMR_a...",Metabolomics
3,-0.227222,-0.227222,1.227221,-0.227222,-0.227222,1.227221,-0.227222,-0.227222,1.227221,-0.227222,...,11.969884,0.0,1000135,train,18,ResidualMultiTaskSurvivalTraining,UKBBSurvivalDatamoduleWithExclusions,"['M_MACE_event', 'M_all_cause_dementia_event',...","['NMR_3hydroxybutyrate', 'NMR_acetate', 'NMR_a...",Metabolomics
4,-0.818723,-0.818723,1.818723,-0.818723,-0.818723,1.818723,-0.818723,-0.818723,1.818723,-0.818723,...,12.446270,0.0,1000161,train,18,ResidualMultiTaskSurvivalTraining,UKBBSurvivalDatamoduleWithExclusions,"['M_MACE_event', 'M_all_cause_dementia_event',...","['NMR_3hydroxybutyrate', 'NMR_acetate', 'NMR_a...",Metabolomics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2595577,-0.728889,-0.728889,1.728889,-0.728889,-0.728889,1.728889,-0.728889,-0.728889,1.728889,-0.728889,...,13.327858,0.0,6024939,test,21,ResidualMultiTaskSurvivalTraining,UKBBSurvivalDatamoduleWithExclusions,"['M_MACE_event', 'M_all_cause_dementia_event',...","['NMR_3hydroxybutyrate', 'NMR_acetate', 'NMR_a...",Metabolomics
2595578,-0.776245,-0.776245,1.776245,-0.776245,-0.776245,1.776245,-0.776245,-0.776245,1.776245,-0.776245,...,12.139630,0.0,6024995,test,21,ResidualMultiTaskSurvivalTraining,UKBBSurvivalDatamoduleWithExclusions,"['M_MACE_event', 'M_all_cause_dementia_event',...","['NMR_3hydroxybutyrate', 'NMR_acetate', 'NMR_a...",Metabolomics
2595579,-0.051542,-0.051542,1.051542,-0.051542,-0.051542,1.051542,-0.051542,-0.051542,1.051542,-0.051542,...,11.685147,0.0,6025042,test,21,ResidualMultiTaskSurvivalTraining,UKBBSurvivalDatamoduleWithExclusions,"['M_MACE_event', 'M_all_cause_dementia_event',...","['NMR_3hydroxybutyrate', 'NMR_acetate', 'NMR_a...",Metabolomics
2595580,0.401364,0.401364,0.598636,0.401364,0.401364,0.598636,0.401364,0.401364,0.598636,0.401364,...,11.649555,0.0,6025116,test,21,ResidualMultiTaskSurvivalTraining,UKBBSurvivalDatamoduleWithExclusions,"['M_MACE_event', 'M_all_cause_dementia_event',...","['NMR_3hydroxybutyrate', 'NMR_acetate', 'NMR_a...",Metabolomics


In [22]:
def categorify(preds):
    for col in tqdm(preds.columns.to_list()):
        if preds[col].dtype == "object": preds[col]= preds[col].astype("category")
    return preds
preds_all_cleaned = categorify(preds_all_cleaned)

  0%|          | 0/1856 [00:00<?, ?it/s]

In [23]:
def fix_column_names(df):
    # rename and fix time bugs!!! -> 0_11_Ft -> Ft at t=10 -> fix earlier
    time_fix_map = dict(zip([col for col in df.columns if "ft" in col], [f"ft_{col}" for col in range(1, len([col for col in df.columns if "ft" in col])+1)]))
    df = df.rename(time_fix_map, axis="columns")
    return df

def wide_to_long(df, endpoint):
    preds_endpoints = []
    for endpoint in tqdm(endpoints):
        time_cols = [f"1_1_ft__{endpoint}"]
        temp = df[["eid", 'features', 'split', 'partition', 'module', 'datamodule'] + time_cols].assign(endpoint = endpoint)[["eid", "endpoint", 'features', 'split', 'partition', 'module', 'datamodule'] + time_cols]
        preds_endpoints.append(fix_column_names(temp))
    df_long = pd.concat(preds_endpoints, axis=0).reset_index(drop=True)
    return df_long.rename(columns={"ft_1":"logh"})
    
preds = wide_to_long(preds_all_cleaned, endpoints)

  0%|          | 0/24 [00:00<?, ?it/s]

In [24]:
#endpoint = "M_MACE"; 
events=[endpoint+'_event' for endpoint in endpoints] 
times=[endpoint+'_event_time' for endpoint in endpoints]

In [25]:
preds_wide = preds.assign(endpoint=lambda x: "logh_" + x.endpoint).\
    pivot(["eid", 'features', 'split', 'partition', 'module', 'datamodule'], columns="endpoint", values="logh").reset_index(drop=False)

In [26]:
# get query features to get wide logh
from functools import reduce
def features_to_wide(predictions):
    preds_features = []
    cols_base = ['eid', 'split', 'partition', 'module', 'datamodule']
    cols_loghs = [f"logh_{endpoint}" for endpoint in endpoints]
    for feature_set in preds_wide.features.unique().to_list():
        temp = preds_wide.query("features==@feature_set")[cols_base+cols_loghs]
        temp.columns = cols_base + [f"{col}_{feature_set}" for col in cols_loghs]
        preds_features.append(temp)
    preds_final = reduce(lambda left,right: pd.merge(left,right,on=cols_base), preds_features)
    return preds_final

In [27]:
preds_wide_fs = features_to_wide(preds_wide)
preds_wide_fs

,eid,split,partition,module,datamodule,logh_M_MACE_Metabolomics,logh_M_coronary_heart_disease_Metabolomics,logh_M_cerebral_stroke_Metabolomics,logh_M_peripheral_arterial_disease_Metabolomics,logh_M_atrial_fibrillation_Metabolomics,...,logh_M_parkinsons_disease_Metabolomics,logh_M_lung_cancer_Metabolomics,logh_M_non_melanoma_skin_cancer_Metabolomics,logh_M_colon_cancer_Metabolomics,logh_M_rectal_cancer_Metabolomics,logh_M_prostate_cancer_Metabolomics,logh_M_breast_cancer_Metabolomics,logh_M_cataracts_Metabolomics,logh_M_glaucoma_Metabolomics,logh_M_fractures_Metabolomics
0,1000084,test,5,ResidualMultiTaskSurvivalTraining,UKBBSurvivalDatamoduleWithExclusions,0.486781,-0.014255,-0.357834,-0.314641,0.258471,...,-1.242479,-1.435712,-0.941847,-1.430353,-1.273272,0.118618,0.127723,-0.459110,-0.562298,-0.704490
1,1000084,train,0,ResidualMultiTaskSurvivalTraining,UKBBSurvivalDatamoduleWithExclusions,0.455325,-0.003404,-0.541668,-0.447002,0.178397,...,-1.423843,-1.357323,-0.985009,-1.484024,-1.301710,0.081710,0.110246,-0.417754,-0.625514,-0.400584
2,1000084,train,1,ResidualMultiTaskSurvivalTraining,UKBBSurvivalDatamoduleWithExclusions,0.496148,0.015636,-0.440400,-0.427101,0.202950,...,-1.402089,-1.469780,-1.179840,-1.507909,-1.180585,0.139365,0.110501,-0.361863,-0.555162,-0.551703
3,1000084,train,2,ResidualMultiTaskSurvivalTraining,UKBBSurvivalDatamoduleWithExclusions,0.462614,0.042355,-0.459206,-0.325135,0.193394,...,-1.408900,-1.395537,-1.012100,-1.613259,-1.323691,0.097098,0.092483,-0.441155,-0.569931,-0.469405
4,1000084,train,3,ResidualMultiTaskSurvivalTraining,UKBBSurvivalDatamoduleWithExclusions,0.559177,0.003853,-0.411211,-0.542751,0.220489,...,-1.384861,-1.363120,-1.175157,-1.605593,-1.391980,0.112488,0.108909,-0.475570,-0.405932,-0.513775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2595577,6025182,valid,16,ResidualMultiTaskSurvivalTraining,UKBBSurvivalDatamoduleWithExclusions,0.640511,0.288207,0.263640,0.225390,0.955587,...,-0.989109,-0.286346,-0.658142,-1.065255,-0.774660,0.160673,0.152242,0.213106,-0.216122,-0.386921
2595578,6025182,valid,17,ResidualMultiTaskSurvivalTraining,UKBBSurvivalDatamoduleWithExclusions,0.589635,0.212322,0.300283,0.215728,0.845687,...,-0.942941,-0.123862,-0.587801,-1.164666,-0.900245,0.171945,0.153931,0.277543,-0.250999,-0.424756
2595579,6025182,valid,18,ResidualMultiTaskSurvivalTraining,UKBBSurvivalDatamoduleWithExclusions,0.688031,0.284718,0.266069,0.199348,0.953456,...,-0.786723,-0.242205,-0.716042,-1.017885,-0.890673,0.187046,0.214847,0.258673,-0.252513,-0.493133
2595580,6025182,valid,19,ResidualMultiTaskSurvivalTraining,UKBBSurvivalDatamoduleWithExclusions,0.615870,0.068719,0.203914,0.143372,0.796473,...,-0.999189,-0.366840,-0.788050,-1.223011,-1.068564,0.142960,0.157635,0.155250,-0.302882,-0.410079


In [28]:
preds_wide_fs.to_feather(f"{data_results_path}/loghazards_model_{run}_metabolomics.feather")

In [29]:
preds_wide_fs = pd.read_feather(f"{data_results_path}/loghazards_model_{run}_metabolomics.feather")

In [30]:
data_temp = pd.read_feather(f"{data_post}/data_merged.feather")
eids_dict = {}
for endpoint in tqdm(endpoints):
    eids_incl = data_temp.query(f"NMR_FLAG==True&{endpoint}==False").eid.to_list()
    if endpoint == "M_MACE": eids_incl = data_temp.copy().query(f"NMR_FLAG==True&{endpoint}==False&statins==False").eid.to_list()
    if endpoint == "M_breast_cancer": eids_incl = data_temp.copy().query(f"NMR_FLAG==True&{endpoint}==False&sex=='Female'").eid.to_list()
    if endpoint == "M_ovarian_cancer": eids_incl = data_temp.copy().query(f"NMR_FLAG==True&{endpoint}==False&sex=='Female'").eid.to_list()
    if endpoint == "M_uterus_cancer": eids_incl = data_temp.copy().query(f"NMR_FLAG==True&{endpoint}==False&sex=='Female'").eid.to_list()
    if endpoint == "M_prostate_cancer": eids_incl = data_temp.copy().query(f"NMR_FLAG==True&{endpoint}==False&sex=='Male'").eid.to_list()
    print(endpoint, len(eids_incl))
    eids_dict[endpoint] = eids_incl

  0%|          | 0/24 [00:00<?, ?it/s]

M_MACE 92629
M_coronary_heart_disease 108551
M_cerebral_stroke 117342
M_peripheral_arterial_disease 115132
M_atrial_fibrillation 116026
M_heart_failure 113636
M_abdominal_aortic_aneurysm 117826
M_venous_thrombosis 114986
M_type_2_diabetes 111745
M_liver_disease 115468
M_renal_disease 111704
M_asthma 102392
M_chronic_obstructuve_pulmonary_disease 110465
M_all_cause_dementia 117245
M_parkinsons_disease 117764
M_lung_cancer 117896
M_non_melanoma_skin_cancer 115996
M_colon_cancer 117617
M_rectal_cancer 117699
M_prostate_cancer 53393
M_breast_cancer 62009
M_cataracts 114345
M_glaucoma 115934
M_fractures 106392


In [31]:
metabolomics = [
'NMR_3hydroxybutyrate',
 'NMR_acetate',
 'NMR_acetoacetate',
 'NMR_acetone',
 'NMR_alanine',
 'NMR_albumin',
 'NMR_apolipoprotein_a1',
 'NMR_apolipoprotein_b',
 'NMR_average_diameter_for_hdl_particles',
 'NMR_average_diameter_for_ldl_particles',
 'NMR_average_diameter_for_vldl_particles',
 'NMR_cholesterol_in_chylomicrons_and_extremely_large_vldl',
 'NMR_cholesterol_in_idl',
 'NMR_cholesterol_in_large_hdl',
 'NMR_cholesterol_in_large_ldl',
 'NMR_cholesterol_in_large_vldl',
 'NMR_cholesterol_in_medium_hdl',
 'NMR_cholesterol_in_medium_ldl',
 'NMR_cholesterol_in_medium_vldl',
 'NMR_cholesterol_in_small_hdl',
 'NMR_cholesterol_in_small_ldl',
 'NMR_cholesterol_in_small_vldl',
 'NMR_cholesterol_in_very_large_hdl',
 'NMR_cholesterol_in_very_large_vldl',
 'NMR_cholesterol_in_very_small_vldl',
 'NMR_cholesteryl_esters_in_chylomicrons_and_extremely_large_vldl',
 'NMR_cholesteryl_esters_in_hdl',
 'NMR_cholesteryl_esters_in_idl',
 'NMR_cholesteryl_esters_in_ldl',
 'NMR_cholesteryl_esters_in_large_hdl',
 'NMR_cholesteryl_esters_in_large_ldl',
 'NMR_cholesteryl_esters_in_large_vldl',
 'NMR_cholesteryl_esters_in_medium_hdl',
 'NMR_cholesteryl_esters_in_medium_ldl',
 'NMR_cholesteryl_esters_in_medium_vldl',
 'NMR_cholesteryl_esters_in_small_hdl',
 'NMR_cholesteryl_esters_in_small_ldl',
 'NMR_cholesteryl_esters_in_small_vldl',
 'NMR_cholesteryl_esters_in_vldl',
 'NMR_cholesteryl_esters_in_very_large_hdl',
 'NMR_cholesteryl_esters_in_very_large_vldl',
 'NMR_cholesteryl_esters_in_very_small_vldl',
 'NMR_citrate',
 'NMR_clinical_ldl_cholesterol',
 'NMR_concentration_of_chylomicrons_and_extremely_large_vldl_particles',
 'NMR_concentration_of_hdl_particles',
 'NMR_concentration_of_idl_particles',
 'NMR_concentration_of_ldl_particles',
 'NMR_concentration_of_large_hdl_particles',
 'NMR_concentration_of_large_ldl_particles',
 'NMR_concentration_of_large_vldl_particles',
 'NMR_concentration_of_medium_hdl_particles',
 'NMR_concentration_of_medium_ldl_particles',
 'NMR_concentration_of_medium_vldl_particles',
 'NMR_concentration_of_small_hdl_particles',
 'NMR_concentration_of_small_ldl_particles',
 'NMR_concentration_of_small_vldl_particles',
 'NMR_concentration_of_vldl_particles',
 'NMR_concentration_of_very_large_hdl_particles',
 'NMR_concentration_of_very_large_vldl_particles',
 'NMR_concentration_of_very_small_vldl_particles',
 'NMR_creatinine',
 'NMR_degree_of_unsaturation',
 'NMR_docosahexaenoic_acid',
 'NMR_free_cholesterol_in_chylomicrons_and_extremely_large_vldl',
 'NMR_free_cholesterol_in_hdl',
 'NMR_free_cholesterol_in_idl',
 'NMR_free_cholesterol_in_ldl',
 'NMR_free_cholesterol_in_large_hdl',
 'NMR_free_cholesterol_in_large_ldl',
 'NMR_free_cholesterol_in_large_vldl',
 'NMR_free_cholesterol_in_medium_hdl',
 'NMR_free_cholesterol_in_medium_ldl',
 'NMR_free_cholesterol_in_medium_vldl',
 'NMR_free_cholesterol_in_small_hdl',
 'NMR_free_cholesterol_in_small_ldl',
 'NMR_free_cholesterol_in_small_vldl',
 'NMR_free_cholesterol_in_vldl',
 'NMR_free_cholesterol_in_very_large_hdl',
 'NMR_free_cholesterol_in_very_large_vldl',
 'NMR_free_cholesterol_in_very_small_vldl',
 'NMR_glucose',
 'NMR_glutamine',
 'NMR_glycine',
 'NMR_glycoprotein_acetyls',
 'NMR_hdl_cholesterol',
 'NMR_histidine',
 'NMR_isoleucine',
 'NMR_ldl_cholesterol',
 'NMR_lactate',
 'NMR_leucine',
 'NMR_linoleic_acid',
 'NMR_monounsaturated_fatty_acids',
 'NMR_omega3_fatty_acids',
 'NMR_omega6_fatty_acids',
 'NMR_phenylalanine',
 'NMR_phosphatidylcholines',
 'NMR_phosphoglycerides',
 'NMR_phospholipids_in_chylomicrons_and_extremely_large_vldl',
 'NMR_phospholipids_in_hdl',
 'NMR_phospholipids_in_idl',
 'NMR_phospholipids_in_ldl',
 'NMR_phospholipids_in_large_hdl',
 'NMR_phospholipids_in_large_ldl',
 'NMR_phospholipids_in_large_vldl',
 'NMR_phospholipids_in_medium_hdl',
 'NMR_phospholipids_in_medium_ldl',
 'NMR_phospholipids_in_medium_vldl',
 'NMR_phospholipids_in_small_hdl',
 'NMR_phospholipids_in_small_ldl',
 'NMR_phospholipids_in_small_vldl',
 'NMR_phospholipids_in_vldl',
 'NMR_phospholipids_in_very_large_hdl',
 'NMR_phospholipids_in_very_large_vldl',
 'NMR_phospholipids_in_very_small_vldl',
 'NMR_polyunsaturated_fatty_acids',
 'NMR_pyruvate',
 'NMR_remnant_cholesterol_nonhdl_nonldl_cholesterol',
 'NMR_saturated_fatty_acids',
 'NMR_sphingomyelins',
 'NMR_total_cholesterol',
 'NMR_total_cholesterol_minus_hdlc',
 'NMR_total_cholines',
 'NMR_total_concentration_of_branchedchain_amino_acids_leucine_isoleucine_valine',
 'NMR_total_concentration_of_lipoprotein_particles',
 'NMR_total_esterified_cholesterol',
 'NMR_total_fatty_acids',
 'NMR_total_free_cholesterol',
 'NMR_total_lipids_in_chylomicrons_and_extremely_large_vldl',
 'NMR_total_lipids_in_hdl',
 'NMR_total_lipids_in_idl',
 'NMR_total_lipids_in_ldl',
 'NMR_total_lipids_in_large_hdl',
 'NMR_total_lipids_in_large_ldl',
 'NMR_total_lipids_in_large_vldl',
 'NMR_total_lipids_in_lipoprotein_particles',
 'NMR_total_lipids_in_medium_hdl',
 'NMR_total_lipids_in_medium_ldl',
 'NMR_total_lipids_in_medium_vldl',
 'NMR_total_lipids_in_small_hdl',
 'NMR_total_lipids_in_small_ldl',
 'NMR_total_lipids_in_small_vldl',
 'NMR_total_lipids_in_vldl',
 'NMR_total_lipids_in_very_large_hdl',
 'NMR_total_lipids_in_very_large_vldl',
 'NMR_total_lipids_in_very_small_vldl',
 'NMR_total_phospholipids_in_lipoprotein_particles',
 'NMR_total_triglycerides',
 'NMR_triglycerides_in_chylomicrons_and_extremely_large_vldl',
 'NMR_triglycerides_in_hdl',
 'NMR_triglycerides_in_idl',
 'NMR_triglycerides_in_ldl',
 'NMR_triglycerides_in_large_hdl',
 'NMR_triglycerides_in_large_ldl',
 'NMR_triglycerides_in_large_vldl',
 'NMR_triglycerides_in_medium_hdl',
 'NMR_triglycerides_in_medium_ldl',
 'NMR_triglycerides_in_medium_vldl',
 'NMR_triglycerides_in_small_hdl',
 'NMR_triglycerides_in_small_ldl',
 'NMR_triglycerides_in_small_vldl',
 'NMR_triglycerides_in_vldl',
 'NMR_triglycerides_in_very_large_hdl',
 'NMR_triglycerides_in_very_large_vldl',
 'NMR_triglycerides_in_very_small_vldl',
 'NMR_tyrosine',
 'NMR_vldl_cholesterol',
 'NMR_valine'
]

In [32]:
AgeSex = ["age_at_recruitment", "sex"]

ASCVDnoblood = [
    "age_at_recruitment", 
    "sex",
    #"cholesterol", 
   # "hdl_cholesterol", 
    "systolic_blood_pressure", 
    "antihypertensives", 
    "diabetes2", 
    "smoking_status_2.0" # current smoker
] 

ASCVD = [
    "age_at_recruitment", 
    "sex",
    "cholesterol", 
    "hdl_cholesterol", 
    "systolic_blood_pressure", 
    "antihypertensives", 
    "diabetes2", 
    "smoking_status_2.0" # current smoker
] 

PANELnoblood = [ # 38 clinical predictors
    # basics
    "age_at_recruitment", 
    "sex", 
    'smoking_status_2.0', # current smoker
    'alcohol_intake_frequency_2.0', # 'Daily or almost daily'
    "daily_physical_activity",
    "education_years",
    "daily_healthy_food",
    
    # family history
    "fh_diabetes",
    
    # diagnoses
    "diabetes2",
        
    # physical
    "body_mass_index_bmi", 
    'waist_hip_ratio',
    "waist_circumference",
    "weight", 
    "standing_height", 
    "systolic_blood_pressure", 
    
    # medications
    'antihypertensives'
        ] 

PANELjustbloodcount = [ # 38 clinical predictors
    # basics
    "age_at_recruitment", 
    "sex", 
    'smoking_status_2.0', # current smoker
    'alcohol_intake_frequency_2.0', # 'Daily or almost daily'
    "daily_physical_activity",
    "education_years",
    "daily_healthy_food",
    
    # family history
    "fh_diabetes",
    
    # diagnoses
    "diabetes2",
        
    # physical
    "body_mass_index_bmi", 
    'waist_hip_ratio',
    "waist_circumference",
    "weight", 
    "standing_height", 
    "systolic_blood_pressure", 
        
    # Blood counts
    'red_blood_cell_erythrocyte_count',
    'white_blood_cell_leukocyte_count',
    'platelet_count',
    'haemoglobin_concentration',
    'haematocrit_percentage',
    'mean_corpuscular_haemoglobin',
    'mean_corpuscular_volume',
    'mean_corpuscular_haemoglobin_concentration',
    
    # medications
    'antihypertensives'
        ] 

PANEL = [ # 38 clinical predictors
    # basics
    "age_at_recruitment", 
    "sex", 
    'smoking_status_2.0', # current smoker
    'alcohol_intake_frequency_2.0', # 'Daily or almost daily'
    "daily_physical_activity",
    "education_years",
    "daily_healthy_food",
    
    # family history
    "fh_diabetes",
    
    # diagnoses
    "diabetes2",
        
    # physical
    "body_mass_index_bmi", 
    'waist_hip_ratio',
    "waist_circumference",
    "weight", 
    "standing_height", 
    "systolic_blood_pressure", 
    
    # lipids
    "cholesterol", 
    "ldl_direct", 
    "hdl_cholesterol",
    "triglycerides",
    
    # diabetes
    'glucose',
    'glycated_haemoglobin_hba1c',
    
    # kidney
    'creatinine',
    'cystatin_c',
    'urea',
    'urate',
        
    # liver
    'aspartate_aminotransferase',
    'alanine_aminotransferase',
    'alkaline_phosphatase',
    'albumin',
    
    # inflammation
    'creactive_protein',
    
    # Blood counts
    'red_blood_cell_erythrocyte_count',
    'white_blood_cell_leukocyte_count',
    'platelet_count',
    'haemoglobin_concentration',
    'haematocrit_percentage',
    'mean_corpuscular_haemoglobin',
    'mean_corpuscular_volume',
    'mean_corpuscular_haemoglobin_concentration',
    
    # medications
    'antihypertensives'
        ] 



In [33]:
# COX + logh(Metabolomics) + (logh(AgeSexMetabolomics))

In [34]:
%env MKL_NUM_THREADS=1
%env NUMEXPR_NUM_THREADS=1
%env OMP_NUM_THREADS=1

env: MKL_NUM_THREADS=1
env: NUMEXPR_NUM_THREADS=1
env: OMP_NUM_THREADS=1


In [41]:
ray.shutdown()

NameError: name 'ray' is not defined

In [36]:
import ray

ray.init(num_cpus=30)#, dashboard_port=24762, dashboard_host="0.0.0.0", include_dashboard=True)#, webui_url="0.0.0.0"))

2022-01-26 18:32:27,693	INFO services.py:1263 -- View the Ray dashboard at http://127.0.0.1:8267


{'node_ip_address': '10.32.105.2',
 'raylet_ip_address': '10.32.105.2',
 'redis_address': '10.32.105.2:54906',
 'object_store_address': '/tmp/ray/session_2022-01-26_18-32-16_596555_3561587/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-01-26_18-32-16_596555_3561587/sockets/raylet',
 'webui_url': '127.0.0.1:8267',
 'session_dir': '/tmp/ray/session_2022-01-26_18-32-16_596555_3561587',
 'metrics_export_port': 52635,
 'node_id': '9365524984c51de375ddbf5f3deb32cc542ea82b435026c45f913198'}

In [37]:
@ray.remote
def get_feather(data_post, partition, split):
    return pd.read_feather(f"{data_post}/partition_{partition}/{split}/data_imputed_normalized.feather").assign(partition=partition, split=split)
    
def get_data():
    return [get_feather.remote(data_post, partition, split) for split in ["train", "valid", "test"] for partition in tqdm(range(22))]

In [38]:
%time
data_raw = pd.concat(ray.get(get_data()), axis=0)

CPU times: user 3 µs, sys: 4 µs, total: 7 µs
Wall time: 13.6 µs


  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

## Add metabolomics PCA

In [39]:
def pca_transform(data_split, split, pca, pca_cols, metabolomics):
    pca_X = pca.transform(data_split[metabolomics].to_numpy())
    return pd.DataFrame(pca_X, columns=pca_cols, index=data_split.eid).assign(split=split)
    
@ray.remote
def fit_pca_partition(data_partition, partition, metabolomics, n_components):
    pca_cols = [f"NMR_PCA{component}" for component in range(n_components)]
    pca = PCA(n_components=n_components)
    x_train = data_partition.query("split=='train'")[metabolomics].to_numpy()
    pca.fit(x_train)
    print(pca.explained_variance_ratio_)
    
    pcas_splits = []
    for split in ["train", "valid", "test"]:
        data_split = data_partition.query("split==@split")
        pcas_splits.append(pca_transform(data_split, split, pca, pca_cols, metabolomics).assign(partition=partition))

    return pd.concat(pcas_splits, axis=0)

In [40]:
from sklearn.decomposition import PCA
n_components = 10
pcas_partitions = []
for partition in tqdm([int(p) for p in partitions]):
    data_partition = data_raw.query("partition==@partition")
    pcas_partitions.append(fit_pca_partition.remote(data_partition, partition, metabolomics, n_components))

  0%|          | 0/22 [00:00<?, ?it/s]

In [41]:
pca_data = pd.concat(ray.get(pcas_partitions), axis=0)

(pid=3586375) [0.45099752 0.22611181 0.10343483 0.04863723 0.02701288 0.01713389
(pid=3586375)  0.01449942 0.01160172 0.01086579 0.00874317]
(pid=3586376) [0.4512137  0.22591627 0.10344533 0.04875616 0.02703755 0.01710045
(pid=3586376)  0.0145502  0.01160729 0.01086778 0.0087483 ]
(pid=3586393) [0.45132636 0.22618433 0.10327056 0.04865323 0.02699177 0.01710906
(pid=3586393)  0.01451339 0.01162043 0.01084798 0.00875132]
(pid=3586377) [0.45134921 0.22579601 0.10344391 0.04862585 0.02707917 0.01712306
(pid=3586377)  0.01449505 0.01159451 0.0108606  0.00874238]
(pid=3586391) [0.45098835 0.22600042 0.10343128 0.04872848 0.0270499  0.01713063
(pid=3586391)  0.01449651 0.01162193 0.01086452 0.00874352]
(pid=3586379) [0.45089657 0.22608252 0.10347927 0.04864923 0.02705751 0.01712599
(pid=3586379)  0.01450911 0.01159973 0.01085067 0.00875854]
(pid=3586381) [0.45089979 0.22610557 0.10337235 0.04868148 0.02705916 0.01712277
(pid=3586381)  0.01447607 0.01163391 0.01087034 0.00874405]
(pid=3586374)

In [42]:
pca_data = pca_data.reset_index().set_index(["partition", "split", "eid"])
data_raw = data_raw.set_index(["partition", "split", "eid"])
preds_wide_fs = preds_wide_fs.set_index(["partition", "split", "eid"])

In [43]:
data_raw_pca = data_raw.merge(pca_data, left_index=True, right_index=True, how="left")

In [44]:
data_raw_us = data_raw_pca.merge(preds_wide_fs, left_index=True, right_index=True, how="left").reset_index()

## Add extra data

In [45]:
decoded = pd.read_feather(f"{base_path}/data/1_decoded/ukb_data_210517.feather")

In [46]:
from sklearn.preprocessing import StandardScaler
extra_data = decoded[["eid", 'waist_circumference_f48_0_0', 'hip_circumference_f49_0_0', 'duration_of_moderate_activity_f894_0_0', 'duration_of_vigorous_activity_f914_0_0']]

extra_data['waist_circumference_f48_0_0'] = extra_data['waist_circumference_f48_0_0'].fillna(extra_data['waist_circumference_f48_0_0'].mean())
extra_data['hip_circumference_f49_0_0'] = extra_data['hip_circumference_f49_0_0'].fillna(extra_data['hip_circumference_f49_0_0'].mean())

extra_data['duration_of_moderate_activity_f894_0_0'] = extra_data['duration_of_moderate_activity_f894_0_0'].fillna(0).replace(-1, 0).replace(-3, 0)
extra_data['duration_of_vigorous_activity_f914_0_0'] = extra_data['duration_of_vigorous_activity_f914_0_0'].fillna(0).replace(-1, 0).replace(-3, 0)

#extra_data["waist_circumference"] = extra_data['waist_circumference_f48_0_0']
extra_data["waist_hip_ratio"] = extra_data['waist_circumference_f48_0_0']/extra_data['hip_circumference_f49_0_0']                                           
extra_data["daily_physical_activity"] = extra_data['duration_of_moderate_activity_f894_0_0']+extra_data['duration_of_vigorous_activity_f914_0_0']

In [47]:
score_data = pd.read_feather(f"{data_results_path}/add_data_211005.feather")
extra_data = extra_data.set_index("eid").merge(score_data.set_index("eid"), left_index=True, right_index=True, how="left").reset_index()

In [48]:
extra_data = extra_data[["eid"]+["waist_hip_ratio", "daily_physical_activity", "education_years", "daily_healthy_food"]]
extra_data

,eid,waist_hip_ratio,daily_physical_activity,education_years,daily_healthy_food
0,1000018,0.794393,10.0,NaN,NaN
1,1000020,0.930085,20.0,NaN,NaN
2,1000037,0.901786,0.0,NaN,NaN
3,1000043,0.942308,30.0,NaN,NaN
4,1000051,0.953488,0.0,NaN,NaN
...,...,...,...,...,...
502471,6025165,0.787234,10.0,NaN,NaN
502472,6025173,0.899083,0.0,NaN,NaN
502473,6025182,1.038462,40.0,13.0,1.0
502474,6025198,1.018519,0.0,NaN,NaN


In [49]:
extra_data.to_feather(f"{data_results_path}/extra_data_220126.feather")

In [50]:
data_raw_us = data_raw_us.set_index("eid").merge(extra_data.set_index("eid"), left_index=True, right_index=True, how="left").reset_index()

In [62]:
1+1

2

Iteration 5: norm_delta = 0.00621, step_size = 0.8450, log_lik = -78277.21060, newton_decrement = 0.12891, seconds_since_start = 1.2
Iteration 7: norm_delta = 0.00000, step_size = 1.0000, log_lik = -76107.59992, newton_decrement = 0.00000, seconds_since_start = 1.7
(pid=3586391) Convergence success after 7 iterations.
Iteration 2: norm_delta = 0.21174, step_size = 0.5000, log_lik = -72794.66273, newton_decrement = 145.35201, seconds_since_start = 0.5
Iteration 4: norm_delta = 0.04628, step_size = 0.6500, log_lik = -79969.90922, newton_decrement = 5.05182, seconds_since_start = 1.0
Iteration 5: norm_delta = 0.00792, step_size = 0.8450, log_lik = -78187.71464, newton_decrement = 0.14126, seconds_since_start = 1.3
Iteration 6: norm_delta = 0.00002, step_size = 1.0000, log_lik = -77145.05207, newton_decrement = 0.00000, seconds_since_start = 1.5
Iteration 6: norm_delta = 0.00003, step_size = 1.0000, log_lik = -77553.62652, newton_decrement = 0.00000, seconds_since_start = 1.5
Iteration 2: 

## Standardize logh and extra data

In [51]:
def norm_variables(data_partition, variables):
    norm = StandardScaler(with_mean=True, with_std=True, copy=True).fit(data_partition.query("split=='train'")[variables].values)

    data_partition[variables] = norm.transform(data_partition[variables].values)
    
    return data_partition

def norm_logh_and_extra(data_all, variables):
    data_new = data_all.copy()
    dfs = []
    for partition in tqdm([int(p) for p in partitions]):
        data_partition = data_new.query("partition==@partition")
        dfs.append(norm_variables(data_partition, variables))
    return pd.concat(dfs, axis=0)

In [52]:
variables_to_norm = ["waist_hip_ratio", "daily_physical_activity", "education_years"] + [col for col in data_raw_us.columns if "logh" and "_Metabolomics" in col]
data_all = norm_logh_and_extra(data_raw_us, variables_to_norm)

  0%|          | 0/22 [00:00<?, ?it/s]

In [53]:
data_all = data_all.reset_index()

In [54]:
data_all.to_feather(f"{data_results_path}/data_all_COX_{run}_metabolomics.feather")

# Train COX

In [55]:
modules = ["PCA", "COX", "DS"]

In [56]:
partitions=data_all.partition.unique().tolist()

In [57]:
data_train = data_all.query("split=='train'") 
data_test = data_all.query("split=='test'")

In [58]:
version=f"COX_{run}"
dump_path = f"{data_post}/{version}"
pathlib.Path(dump_path).mkdir(parents=True, exist_ok=True)

In [76]:
from lifelines import CoxPHFitter
from dask.diagnostics import ProgressBar
from lifelines.exceptions import ConvergenceError
import pickle

def flatten_dict(features):
    df = pd.json_normalize(d, sep='_')
    t = list(df.to_dict(orient='records')[0].values())
    flat_list = [item for sublist in t for item in sublist]
    return list(set(flat_list))

def flatten_list(l):
    return [item for sublist in l for item in sublist]

def get_all_features(metabolomics, endpoints):
    f_dicts = flatten_list([[ds for ds in get_features(metabolomics, endpoint).values()] for endpoint in endpoints])
    fs = list(sorted(set(flatten_list(flatten_list([list(p.values()) for p in f_dicts])))))
    return fs + [f"{endpoint}_event" for endpoint in endpoints] + [f"{endpoint}_event_time" for endpoint in endpoints]

def get_features(metabolomics, endpoint):
    features = {"COX": {"Metabolomics": metabolomics,
                        "Age+Sex": AgeSex, 
                        "ASCVDnoblood": ASCVDnoblood,
                        "ASCVD": ASCVD,
                        "PANELnoblood": PANELnoblood,
                        "PANELjustbloodcount": PANELjustbloodcount,
                        "PANEL": PANEL,
                       },
                "PCA": {"Metabolomics": [f'NMR_PCA{i}' for i in range(10)]},
                
                "DS":  {"Metabolomics": [f"logh_{endpoint}_Metabolomics"], 
                        "Age+Sex+Metabolomics": AgeSex+[f"logh_{endpoint}_Metabolomics"], 
                        "ASCVDnoblood+Metabolomics": ASCVDnoblood+[f"logh_{endpoint}_Metabolomics"],
                         "ASCVD+Metabolomics": ASCVD+[f"logh_{endpoint}_Metabolomics"],
                        "PANELnoblood+Metabolomics": PANELnoblood+[f"logh_{endpoint}_Metabolomics"],
                        "PANELjustbloodcount+Metabolomics": PANELjustbloodcount+[f"logh_{endpoint}_Metabolomics"],
                        "PANEL+Metabolomics": PANEL+[f"logh_{endpoint}_Metabolomics"],
                       }
               }
    
    return features
    
def fit_cox(train, endpoint, penalizer, step_size):
    cph = CoxPHFitter(penalizer=penalizer)
    cph.fit(train, f"{endpoint}_event_time", f"{endpoint}_event", step_size=step_size, show_progress=True)
    return cph

@ray.remote
def fit_partition(train_partition, endpoint, module, features, partition):
    for feature_set, covariates in tqdm(features[module].items()):
        cph_path = f"{dump_path}/{module}_{endpoint}_{feature_set}_{partition}.p"
        if endpoint == "M_type_2_diabetes":
            if "diabetes1" in covariates: covariates = [c for c in covariates if c!="diabetes1"]
            if "diabetes2" in covariates: covariates = [c for c in covariates if c!="diabetes2"]
        if endpoint in ["M_prostate_cancer", "M_ovarian_cancer", "M_breast_cancer", "M_uterus_cancer"]:
            if "sex" in covariates: covariates = [c for c in covariates if c!="sex"]
        if not os.path.isfile(cph_path):
            try:
                cph = fit_cox(train_partition[covariates + [f"{endpoint}_event", f"{endpoint}_event_time"]], endpoint, penalizer=0.0, step_size=0.5)
                pickle.dump(cph, open(cph_path, "wb"))
            except ConvergenceError:
                print("ConvergenceError", module, endpoint, feature_set, partition, "problem: reduce step size")
                try:
                    cph = fit_cox(train_partition[covariates + [f"{endpoint}_event", f"{endpoint}_event_time"]], endpoint, penalizer=0.0, step_size=0.1)
                    pickle.dump(cph, open(cph_path, "wb"))
                    print("ConvergenceError", module, endpoint, feature_set, partition, "trying with reduced step size ... successfull")
                except:
                    print("ConvergenceError", module, endpoint, feature_set, partition, "trying with reduced step size ... failed")
    return True

def train_cox(endpoints, modules, partitions, data_train):
    frame_dict = {}
    features_all = get_all_features(metabolomics, endpoints)
    data_train = data_train[["eid", "partition", "split"]+features_all]
    with ProgressBar():   
        for module in tqdm(modules):
            for endpoint in tqdm(endpoints):
                features = get_features(metabolomics, endpoint)
                eids_incl = eids_dict[endpoint] 
                train_endpoint = data_train.query("eid==@eids_incl")
                for partition in partitions:
                    train_partition = train_endpoint.query("partition==@partition")
                    frame_dict[f"{module}_{endpoint}_{partition}"] = fit_partition.remote(train_partition, endpoint, module, features, partition)
                    del train_partition
    return frame_dict

In [56]:
ray.shutdown()

In [ ]:
endpoints

In [ ]:
frame_dict = train_cox(endpoints, modules, partitions, data_train)

In [ ]:
ray.shutdown()

## Apply COX to data

In [91]:
import joblib
@ray.remote
def get_cph(path): 
    with open(path,'rb') as f:
        cph = pickle.load(f)
    return cph#return pd.read_csv(f"{path[:-8]}.csv", index_col=0)

In [211]:
ray.shutdown()

In [79]:
import glob, os
cph_dict = {}
for path in tqdm(glob.glob(f"{dump_path}/*.p")):
    cph_dict[pathlib.Path(path).stem] = get_cph.remote(path)

Iteration 7: norm_delta = 0.00000, step_size = 1.0000, log_lik = -79990.68336, newton_decrement = 0.00000, seconds_since_start = 1.6
(pid=3586374) Convergence success after 7 iterations.
Iteration 7: norm_delta = 0.00000, step_size = 1.0000, log_lik = -80246.82578, newton_decrement = 0.00000, seconds_since_start = 1.5
(pid=3586392) Convergence success after 7 iterations.
Iteration 7: norm_delta = 0.00000, step_size = 1.0000, log_lik = -80609.34910, newton_decrement = 0.00000, seconds_since_start = 1.6
(pid=3586384) Convergence success after 7 iterations.


  0%|          | 0/9504 [00:00<?, ?it/s]

In [80]:
for path in tqdm(glob.glob(f"{dump_path}/*.p")):
    cph_dict[pathlib.Path(path).stem] = ray.get(cph_dict[pathlib.Path(path).stem])

  0%|          | 0/9504 [00:00<?, ?it/s]

In [81]:
def predict_cox(endpoints, modules, partitions, data_test, cph_dict):
    times = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
    time_cols = {t: f"Ft_{t}" for t in times}
    
    results = []
    test_preds = []
    for module in tqdm(modules):
        for endpoint in tqdm(endpoints):
            features = get_features(metabolomics, endpoint)
            eids_incl = eids_dict[endpoint] 
            test_endpoint = data_test.query("eid==@eids_incl")
            for partition in partitions:
                test_partition = test_endpoint.query("partition==@partition")
                for feature_set, covariates in features[module].items():
                    #print(endpoint, feature_set)
                    if endpoint == "M_type_2_diabetes":
                        if "diabetes1" in covariates: covariates = [c for c in covariates if c!="diabetes1"]
                        if "diabetes2" in covariates: covariates = [c for c in covariates if c!="diabetes2"]
                    if endpoint in ["M_prostate_cancer", "M_ovarian_cancer", "M_breast_cancer", "M_uterus_cancer"]:
                        if "sex" in covariates: covariates = [c for c in covariates if c!="sex"]
                    cph = cph_dict[f"{module}_{endpoint}_{feature_set}_{partition}"]
                    surv_test = 1-cph.predict_survival_function(test_partition[covariates+ [f"{endpoint}_event", f"{endpoint}_event_time"]], times=times) 
                    temp_pred = test_partition.reset_index()[["eid"]].assign(endpoint=endpoint, module=module, features=feature_set, partition=partition)
                    for t, col in time_cols.items(): temp_pred[col] = surv_test.T[t].to_list()
                    test_preds.append(temp_pred)
                    results.append({"endpoint": endpoint,
                                    "module": module,
                                    "features": feature_set,
                                    "HR_dict": cph.hazard_ratios_.to_dict(),
                                    "partition": partition,
                                    "cph": cph,
                                   })
    return results, test_preds

In [82]:
results, test_preds = predict_cox(endpoints, modules, partitions, data_test, cph_dict)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

## Save Predictions

In [83]:
predictions = pd.concat(test_preds, axis=0).reset_index()

In [84]:
predictions.to_feather(f"{data_results_path}/predictions_{run}_metabolomics.feather")

In [85]:
predictions.value_counts(["module", "features"])

module  features                        
COX     ASCVD                               2604192
        ASCVDnoblood                        2604192
        Age+Sex                             2604192
        Metabolomics                        2604192
        PANEL                               2604192
        PANELjustbloodcount                 2604192
        PANELnoblood                        2604192
DS      ASCVD+Metabolomics                  2604192
        ASCVDnoblood+Metabolomics           2604192
        Age+Sex+Metabolomics                2604192
        Metabolomics                        2604192
        PANEL+Metabolomics                  2604192
        PANELjustbloodcount+Metabolomics    2604192
        PANELnoblood+Metabolomics           2604192
PCA     Metabolomics                        2604192
dtype: int64

In [86]:
predictions.value_counts(["endpoint"])

endpoint                               
M_lung_cancer                              1768440
M_abdominal_aortic_aneurysm                1767390
M_parkinsons_disease                       1766460
M_rectal_cancer                            1765485
M_colon_cancer                             1764255
M_cerebral_stroke                          1760130
M_all_cause_dementia                       1758675
M_atrial_fibrillation                      1740390
M_non_melanoma_skin_cancer                 1739940
M_glaucoma                                 1739010
M_liver_disease                            1732020
M_peripheral_arterial_disease              1726980
M_venous_thrombosis                        1724790
M_cataracts                                1715175
M_heart_failure                            1704540
M_type_2_diabetes                          1676175
M_renal_disease                            1675560
M_chronic_obstructuve_pulmonary_disease    1656975
M_coronary_heart_disease                  

## Save COX models

In [87]:
results_df = pd.DataFrame().append(results, ignore_index=True)

In [88]:
results_df

,endpoint,module,features,HR_dict,partition,cph
0,M_MACE,PCA,Metabolomics,"{'NMR_PCA0': 1.024527366554924, 'NMR_PCA1': 0....",0,<lifelines.CoxPHFitter: fitted with 76127 tota...
1,M_MACE,PCA,Metabolomics,"{'NMR_PCA0': 1.0248745683486296, 'NMR_PCA1': 0...",1,<lifelines.CoxPHFitter: fitted with 76087 tota...
2,M_MACE,PCA,Metabolomics,"{'NMR_PCA0': 1.025513360791944, 'NMR_PCA1': 0....",2,<lifelines.CoxPHFitter: fitted with 77184 tota...
3,M_MACE,PCA,Metabolomics,"{'NMR_PCA0': 1.0250266269049448, 'NMR_PCA1': 0...",3,<lifelines.CoxPHFitter: fitted with 77807 tota...
4,M_MACE,PCA,Metabolomics,"{'NMR_PCA0': 1.0254650552953355, 'NMR_PCA1': 0...",4,<lifelines.CoxPHFitter: fitted with 77842 tota...
...,...,...,...,...,...,...
7915,M_fractures,DS,ASCVDnoblood+Metabolomics,"{'age_at_recruitment': 1.2443299077749335, 'se...",21,<lifelines.CoxPHFitter: fitted with 95702 tota...
7916,M_fractures,DS,ASCVD+Metabolomics,"{'age_at_recruitment': 1.2456859181929998, 'se...",21,<lifelines.CoxPHFitter: fitted with 95702 tota...
7917,M_fractures,DS,PANELnoblood+Metabolomics,"{'age_at_recruitment': 1.2400313359465613, 'se...",21,<lifelines.CoxPHFitter: fitted with 95702 tota...
7918,M_fractures,DS,PANELjustbloodcount+Metabolomics,"{'age_at_recruitment': 1.2401629760934452, 'se...",21,<lifelines.CoxPHFitter: fitted with 95702 tota...


In [89]:
def get_hr(hr_dict, endpoint): 
    if f"logh_{endpoint}_Metabolomics" in hr_dict: 
        hr = hr_dict[f"logh_{endpoint}_Metabolomics"]
    else:
        hr = np.nan
    return hr

In [90]:
import pickle
results_df["HR_metabolomics"] = [get_hr(hr_dict, endpoint) for endpoint, hr_dict in zip(results_df["endpoint"], results_df["HR_dict"])]
results_df.drop(columns=["cph"]).to_feather(f"{data_results_path}/cox_{run}_metabolomics.feather")

In [74]:
#!ls -lah $data_results_path
!ls -lah $dump_path | tail

-rwxrwx---  1 steinfej posix-nogroup 6.9M Oct  6 15:04 PCA_M_venous_thrombosis_Metabolomics_2.p
-rwxrwx---  1 steinfej posix-nogroup 7.4M Oct  6 15:04 PCA_M_venous_thrombosis_Metabolomics_20.p
-rwxrwx---  1 steinfej posix-nogroup 7.4M Oct  6 15:04 PCA_M_venous_thrombosis_Metabolomics_21.p
-rwxrwx---  1 steinfej posix-nogroup 6.9M Oct  6 15:04 PCA_M_venous_thrombosis_Metabolomics_3.p
-rwxrwx---  1 steinfej posix-nogroup 6.9M Oct  6 15:04 PCA_M_venous_thrombosis_Metabolomics_4.p
-rwxrwx---  1 steinfej posix-nogroup 7.0M Oct  6 15:04 PCA_M_venous_thrombosis_Metabolomics_5.p
-rwxrwx---  1 steinfej posix-nogroup 7.0M Oct  6 15:04 PCA_M_venous_thrombosis_Metabolomics_6.p
-rwxrwx---  1 steinfej posix-nogroup 7.0M Oct  6 15:04 PCA_M_venous_thrombosis_Metabolomics_7.p
-rwxrwx---  1 steinfej posix-nogroup 7.0M Oct  6 15:04 PCA_M_venous_thrombosis_Metabolomics_8.p
-rwxrwx---  1 steinfej posix-nogroup 7.0M Oct  6 15:04 PCA_M_venous_thrombosis_Metabolomics_9.p


In [13]:
# STORE HRs in different table


In [87]:
import glob
import pickle
import re

In [88]:
def _extract_logh(d):
    for k, v in d.items():
        if k.startswith('logh_'):
            return v

In [90]:
res

('M_heart_failure', 'Age+Sex+Metabolomics', '6')

In [91]:
# load models and put HRs into a df:
df_dict = {"endpoint": [],
           "partition": [],
           "features": [],
           "HR_Metabolomics": []
          }

for fp in glob.glob(os.path.join(dump_path,"DS_*.p")):
    try:
        res = re.search("(M_[a-zA-Z_]+[2a-zA-Z_]+)_([3a-zA-Z+]+)_(\d+)", fp).groups()

        df_dict['endpoint'].append(res[0])
        df_dict['features'].append(res[1])
        df_dict['partition'].append(res[2])

        cph = pickle.load(open(fp, "rb"))
        HR = _extract_logh(cph.hazard_ratios_.to_dict())
    #     HR = cph.hazard_ratios_.to_dict()[f'logh_{res[0]}_Metabolomics']
        df_dict['HR_Metabolomics'].append(HR)
    except:
        print(fp)

In [93]:
HR_df = pd.DataFrame.from_dict(df_dict, orient="columns")
HR_df['module'] = 'DS'

In [64]:
HR_df.head()

NameError: name 'HR_df' is not defined

In [75]:
HR_df.to_feather(f"{data_results_path}/MET_HRs_210819_metabolomics.feather")

In [44]:
results_df = results_df.assign(HR_logh=results_df['hazard_ratio'].apply(_extract_logh))

In [45]:
results_df['features'].unique()

array(['Age+Sex', 'Metabolomics', 'Age+Sex+Metabolomics',
       'AgeSexMetabolomics'], dtype=object)

In [37]:
results_df[["endpoint", "module", "features", 'HR_logh']].reset_index().to_feather(f"{data_results_path}/MET_HRs_{run}_metabolomics.feather")

In [39]:
f"{data_results_path}/MET_HRs_{run}_metabolomics.feather"

'/data/analysis/ag-reils/ag-reils-shared/cardioRS/results/projects/21_metabolomics_multitask/data/MET_HRs_210821_metabolomics.feather'

In [ ]:
raise NotImplementedError()

In [38]:
# Other stuff

In [ ]:
cph = results_df.set_index(["endpoint", "module", "features"]).loc[('M_MACE', "DS", "Age+Sex+Metabolomics"), "cph"].iloc[0]

In [ ]:
cph.print_summary()

In [ ]:
dir(cph)

In [ ]:
cph.summary["coef"]

In [ ]:
cph.baseline_cumulative_hazard_

In [ ]:
cph.baseline_hazard_

In [ ]:
dir(cph)

In [ ]:
cph.print_summary()

In [ ]:
import plotly.express as px
scores_plot = ["COX_AgeSex", "DeepSurv_AgeSexMetabolomics"]
temp = results_df.assign(score = lambda x: x.module + "_" + x.features).query("score==@scores_plot")
fig = px.scatter(temp, y="cindex", x="score", color="features", facet_col="endpoint", facet_col_wrap=12, template="plotly_dark",
               category_orders={"features": ["COX_Metabolomics", "DeepSurv_Metabolomics", 
                                             "COX_AgeSex", 
                                             "COX_AgeSexMetabolomics", "DeepSurv_AgeSexMetabolomics"]})
fig.update_xaxes(matches=None)
#fig.update_yaxes(matches=None)
fig.show("png", width=1500, height=1000)

In [ ]:
import plotly.express as px
temp = results_df.assign(score = lambda x: x.module + "_" + x.features)
fig = px.scatter(temp, y="cindex", x="score", color="features", facet_col="endpoint", facet_col_wrap=5, template="plotly_dark",
               category_orders={"features": ["COX_Metabolomics", "DeepSurv_Metabolomics", 
                                             "COX_AgeSex", 
                                             "COX_AgeSexMetabolomics", "DeepSurv_AgeSexMetabolomics"]})
fig.update_xaxes(matches=None)
#fig.update_yaxes(matches=None)
fig.show("png", width=1500, height=1000)

In [ ]:
import plotly.express as px
temp = results_df.assign(score = lambda x: x.module + "_" + x.features)
fig = px.scatter(temp, y="cindex", x="score", color="features", facet_col="endpoint", facet_col_wrap=5, template="plotly_dark",
               category_orders={"features": ["COX_Metabolomics", "DeepSurv_Metabolomics", 
                                             "COX_AgeSex", "DeepSurv_AgeSex",
                                             "COX_AgeSexMetabolomics", "DeepSurv_AgeSexMetabolomics"]})
fig.update_xaxes(matches=None)
#fig.update_yaxes(matches=None)
fig.show("png", width=1500, height=1000)

In [ ]:
dpath = "/data/analysis/ag-reils/steinfej/data/3_datasets_post/210730_imaging_visit"
for partition in range(5):
    for split in ["train", "valid", "test"]:
        ddf = pd.read_feather(f"{dpath}/partition_{partition}/{split}/data.feather")
        print(partition, split, len(ddf))

In [ ]:
data_X = pd.read_feather("/data/analysis/ag-reils/ag-reils-shared/cardioRS/data/3_datasets_post/210730_imaging_visit/data_merged.feather")

In [ ]:
data_X.set_index("uk_biobank_assessment_centre_2_0").index.value_counts().index.to_list()

In [ ]:
data_X.set_index("uk_biobank_assessment_centre_2_0").index.value_counts()

In [ ]:
import plotly.express as px
temp = results_df.assign(score = lambda x: x.module + "_" + x.features)
fig = px.scatter(temp, y="cindex", x="score", color="features", facet_col="endpoint", facet_col_wrap=5, template="plotly_dark",
               category_orders={"features": ["COX_Metabolomics", "DeepSurv_Metabolomics", 
                                             "COX_AgeSex", "DeepSurv_AgeSex",
                                             "COX_AgeSexMetabolomics", "DeepSurv_AgeSexMetabolomics"]})
fig.update_xaxes(matches=None)
#fig.update_yaxes(matches=None)
fig.show("png", width=1500, height=1000)

In [ ]:
import plotly.express as px
temp = results_df.assign(score = lambda x: x.module + "_" + x.features)
fig = px.scatter(temp, y="cindex", x="score", color="features", facet_col="endpoint", facet_col_wrap=5, template="plotly_dark",
               category_orders={"features": ["COX_Metabolomics", "DeepSurv_Metabolomics", 
                                             "COX_AgeSex", "DeepSurv_AgeSex",
                                             "COX_AgeSexMetabolomics", "DeepSurv_AgeSexMetabolomics"]})
fig.update_xaxes(matches=None)
#fig.update_yaxes(matches=None)
fig.show("png", width=1500, height=1000)

In [ ]:
import plotly.express as px
temp = results_df.assign(score = lambda x: x.module + "_" + x.features)
fig = px.scatter(temp, y="cindex", x="score", color="features", facet_col="endpoint", facet_col_wrap=5, template="plotly_dark",
               category_orders={"features": ["COX_Metabolomics", "DeepSurv_Metabolomics", 
                                             "COX_AgeSex", "DeepSurv_AgeSex",
                                             "COX_AgeSexMetabolomics", "DeepSurv_AgeSexMetabolomics"]})
fig.update_xaxes(matches=None)
fig.update_yaxes(matches=None)
fig.show("png", width=1500, height=1000)

In [ ]:
import plotly.express as px
temp = results_df.assign(score = lambda x: x.module + "_" + x.features)
fig = px.scatter(temp, y="cindex", x="score", color="features", facet_col="endpoint", facet_col_wrap=5, template="plotly_dark",
               category_orders={"features": ["COX_Metabolomics", "DeepSurv_Metabolomics", 
                                             "COX_AgeSex", "DeepSurv_AgeSex",
                                             "COX_AgeSexMetabolomics", "DeepSurv_AgeSexMetabolomics"]})
fig.update_xaxes(matches=None)
fig.update_yaxes(matches=None)
fig.show("png", width=1500, height=1000)

In [ ]:
import plotly.express as px
temp = results_df.assign(score = lambda x: x.module + "_" + x.features)
fig = px.scatter(temp, y="cindex", x="score", color="features", facet_col="endpoint", facet_col_wrap=5, template="plotly_dark",
               category_orders={"features": ["COX_Metabolomics", "DeepSurv_Metabolomics", 
                                             "COX_AgeSex", "DeepSurv_AgeSex",
                                             "COX_AgeSexMetabolomics", "DeepSurv_AgeSexMetabolomics"]})
fig.update_xaxes(matches=None)
fig.update_yaxes(matches=None)
fig.show("png", width=1500, height=1000)

In [ ]:
import plotly.express as px
temp = results_df.assign(score = lambda x: x.module + "_" + x.features)
fig = px.scatter(temp, y="cindex", x="score", color="features", facet_col="endpoint", facet_col_wrap=5, template="plotly_dark",
               category_orders={"features": ["COX_Metabolomics", "DeepSurv_Metabolomics", 
                                             "COX_AgeSex", "DeepSurv_AgeSex",
                                             "COX_AgeSexMetabolomics", "DeepSurv_AgeSexMetabolomics"]})
fig.update_xaxes(matches=None)
fig.update_yaxes(matches=None)
fig.show("png", width=1500, height=1000)

In [ ]:
import plotly.express as px
temp = results_df.assign(score = lambda x: x.module + "_" + x.features)
fig = px.scatter(temp, y="cindex", x="score", color="features", facet_col="endpoint", facet_col_wrap=5, template="plotly_dark",
               category_orders={"features": ["COX_Metabolomics", "DeepSurv_Metabolomics", 
                                             "COX_AgeSex", "DeepSurv_AgeSex",
                                             "COX_AgeSexMetabolomics", "DeepSurv_AgeSexMetabolomics"]})
fig.update_xaxes(matches=None)
#fig.update_yaxes(matches=None)
fig.show("png", width=1500, height=1000)

In [ ]:
import plotly.express as px
temp = results_df.assign(score = lambda x: x.module + "_" + x.features)
fig = px.violin(temp, y="cindex", x="features", color="features", box=True, points="all", facet_col="endpoint", facet_col_wrap=4,
               category_orders={"features": ["Metabolomics", "AgeSex", "AgeSexMetabolomics"]})
fig.update_xaxes(matches=None)
fig.show("png", width=1500, height=1000)

In [ ]:
preds.to_feather(f"{data_results_path}/predictions_model_210720.feather")

In [ ]:
from lifelines import RegressionFitter

In [ ]:
self._central_values = self._compute_central_values_of_raw_training_data(df, self.strata)
self.baseline_survival_ = self._compute_baseline_survival()
baseline_hazard_ = self._compute_baseline_hazards(predicted_partial_hazards_)
baseline_cumulative_hazard_ = self._compute_baseline_cumulative_hazard(baseline_hazard_)

In [ ]:
cph_semi = CoxPHFitter().fit(rossi, 'week', event_col='arrest')
cph_piecewise = CoxPHFitter(baseline_estimation_method="piecewise", breakpoints=[20, 35]).fit(rossi, 'week', event_col='arrest')

ax = cph_spline.baseline_cumulative_hazard_.plot()